In [1]:
import pandas as pd

In [3]:
import pandas as pd

# Path to your file
file_path = r"D:\FinalYearProject\Matrimony_Matchmaker\data\DHS\IACR7EFL.DTA"

# Use the StataReader to get the data and the labels separately
with pd.io.stata.StataReader(file_path) as reader:
    # 1. Load the data without converting categoricals
    df = reader.read(convert_categoricals=False)
    
    # 2. Extract the value labels (dictionary) if you need them later
    # This stores what 1, 2, 3 mean for each column
    value_labels = reader.value_labels()

# Now the dataframe will load successfully
print("Data loaded successfully!")
print(df.head())

Data loaded successfully!
            caseid v000  v001  v002  v003  v004    v005  v006  v007  v008  \
0    0100103022 04  IA7   130    22     4   130  201224    12  2019  1440   
1    0100103033 02  IA7   130    33     2   130  201224    12  2019  1440   
2    0100100997 02  IA7   109    97     2   109  196628    11  2019  1439   
3    0100100966 02  IA7   109    66     2   109  196628    11  2019  1439   
4    0100100914 02  IA7   109    14     2   109  196628    11  2019  1439   

   ...  smb81  smb305  smb306  sm190s   sm191s  sm190us  sm191us  sm190rs  \
0  ...    0.0    70.1    77.2       5  1264770      NaN      NaN      5.0   
1  ...    0.0    92.5    96.3       2   144090      NaN      NaN      3.0   
2  ...    0.0    72.8    78.2       2   103120      NaN      NaN      3.0   
3  ...    0.0    81.2    84.4       1  -460030      NaN      NaN      2.0   
4  ...    0.0    81.4    84.5       3   626660      NaN      NaN      4.0   

     sm191rs  smcrefuse  
0  1690700.0        Na

In [15]:
df["v131"].unique()

array([991, 992, 993, 998], dtype=int16)

In [16]:
import pandas as pd
import numpy as np

# Load the data raw to avoid Stata label conflicts
file_path = r"D:\FinalYearProject\Matrimony_Matchmaker\data\DHS\IACR7EFL.DTA"
with pd.io.stata.StataReader(file_path) as reader:
    raw_df = reader.read(convert_categoricals=False)

# Standardize column names to lowercase to match the DCT file provided
raw_df.columns = [col.lower() for col in raw_df.columns]

# Helper function to map education years to your specific B.Tech/MBA format
def map_degree(years):
    if pd.isna(years) or years > 90: return "N/A"
    if years >= 17: return "Post-Graduate (MBA/MA)"
    if years >= 15: return "Graduate (B.Tech/B.Sc/BA)"
    if years >= 12: return "Higher Secondary"
    return "Secondary or below"

# Helper function to map profession codes from the labels provided
def map_occ(code):
    occ_map = {0: "Not Working", 1: "Professional/IT/Tech", 3: "Clerical", 4: "Sales", 
               5: "Services", 6: "Agricultural", 7: "Manual"}
    return occ_map.get(code, "Other")

processed_df = pd.DataFrame()

# 1. IDENTIFICATION
processed_df['ID'] = raw_df['v001'].astype(str) + "_" + raw_df['v002'].astype(str)

# 2. FEMALE FEATURES (Respondent)
processed_df['Wife_Age'] = raw_df['v012']
processed_df['Wife_Education'] = raw_df['v133'].apply(map_degree)
processed_df['Wife_Profession'] = raw_df['v717'].apply(map_occ)
processed_df['Wife_Caste'] = raw_df['v131'].map({991: 'Caste', 992: 'Tribe', 993: 'No Caste'})
processed_df['Wife_Religion'] = raw_df['v130'].map({1: 'Hindu', 2: 'Muslim', 3: 'Christian', 4: 'Sikh', 5: 'Buddhist', 6: 'Jain'})
processed_df['Wife_Residence'] = raw_df['v025'].map({1: 'Urban', 2: 'Rural'})

# 3. MALE FEATURES (Husband/Partner)
processed_df['Husband_Age'] = raw_df['v730'] # v730 is Husband's age in the Couple file
processed_df['Husband_Education'] = raw_df['mv133'].apply(map_degree)
processed_df['Husband_Profession'] = raw_df['mv717'].apply(map_occ)
processed_df['Husband_Caste'] = raw_df['mv131'].map({991: 'Caste', 992: 'Tribe', 993: 'No Caste'})
processed_df['Husband_Religion'] = raw_df['mv130'].map({1: 'Hindu', 2: 'Muslim', 3: 'Christian', 4: 'Sikh', 5: 'Buddhist', 6: 'Jain'})
processed_df['Husband_Residence'] = raw_df['mv025'].map({1: 'Urban', 2: 'Rural'})

# 4. SUCCESS METRIC (Domestic Conflict Proxy)
# d105a: "Ever been pushed, shook or had something thrown by husband"
processed_df['Conflict_Score'] = raw_df['d105a'].replace({0: 'No', 1: 'Yes', 2: 'Yes', 3: 'Yes', 4: 'Yes'})

# 5. SAVE
output_path = r"D:\FinalYearProject\Matrimony_Matchmaker\data\DHS\final_couples_profile.csv"
processed_df.to_csv(output_path, index=False)

print(f"Extraction successful! {len(processed_df)} couples processed.")
print(processed_df.head())

Extraction successful! 57693 couples processed.
       ID  Wife_Age      Wife_Education Wife_Profession Wife_Caste  \
0  130_22        29  Secondary or below        Services      Caste   
1  130_33        36  Secondary or below     Not Working      Caste   
2  109_97        48  Secondary or below     Not Working      Tribe   
3  109_66        39  Secondary or below     Not Working      Caste   
4  109_14        31  Secondary or below     Not Working      Tribe   

  Wife_Religion Wife_Residence  Husband_Age   Husband_Education  \
0        Muslim          Rural           31    Higher Secondary   
1        Muslim          Rural           42  Secondary or below   
2        Muslim          Rural           54  Secondary or below   
3        Muslim          Rural           38  Secondary or below   
4        Muslim          Rural           39    Higher Secondary   

  Husband_Profession Husband_Caste Husband_Religion Husband_Residence  \
0           Services         Caste           Muslim    

In [27]:
import pandas as pd
import numpy as np

# Load the raw dataset
# raw_df = pd.read_stata(r"D:\FinalYearProject\Matrimony_Matchmaker\data\DHS\IACR7EFL.DTA", convert_categoricals=False)
# raw_df.columns = [col.lower() for col in raw_df.columns]

# --- 1. Map Columns to their full Dictionary Descriptions ---
# These descriptions are taken exactly from your provided .txt and .dct files

satisfaction_mapping = {
    # Emotional/Verbal
    'd103a': "Ever_been_humiliated_by_husband_partner",
    'd103c': "Ever_been_insulted_or_made_to_feel_bad_by_husband_partner",
    
    # Physical Conflict
    'd105a': "Ever_been_pushed_shook_or_had_something_thrown_by_husband_partner",
    'd105b': "Ever_been_slapped_by_husband_partner",
    
    # Controlling Behavior
    'd101a': "Husband_partner_jealous_if_respondent_talks_with_other_men",
    'd101c': "Husband_partner_does_not_permit_respondent_to_meet_female_friends",
    'd101e': "Husband_partner_insists_on_knowing_where_respondent_is",
    
    # Agency / Decision Making
    'v743a': "Person_who_usually_decides_on_respondents_health_care",
    'v743b': "Person_who_usually_decides_on_large_household_purchases",
    
    # Fear
    'd129': "Respondent_afraid_of_husband_partner_most_of_the_time_sometimes_or_never"
}

# --- 2. Add these to your finalized dataframe ---

# We iterate through the mapping and add the raw values under the descriptive headers
for code, description in satisfaction_mapping.items():
    if code in raw_df.columns:
        processed_df[description] = raw_df[code]

# --- 3. Optional: Convert numeric codes to simple "Yes/No" or "Problem" text 
# for better readability in your final CSV

# For the Conflict/Violence variables (d series), 0 is 'No', >0 is 'Yes'
conflict_cols = [
    "Ever_been_humiliated_by_husband_partner",
    "Ever_been_insulted_or_made_to_feel_bad_by_husband_partner",
    "Ever_been_pushed_shook_or_had_something_thrown_by_husband_partner",
    "Ever_been_slapped_by_husband_partner",
    "Husband_partner_jealous_if_respondent_talks_with_other_men",
    "Husband_partner_does_not_permit_respondent_to_meet_female_friends",
    "Husband_partner_insists_on_knowing_where_respondent_is"
]

for col in conflict_cols:
    if col in processed_df.columns:
        processed_df[col] = np.where(processed_df[col] > 0, "Yes/Problem", "No/Never")

# For Decision Making (v743 series), keep numeric or map based on your labels:
# 1=Wife, 2=Joint, 4=Husband, 5=Other
# No categorization requested, so keeping as is.

# --- 4. Save the finalized file ---
output_path = r"D:\FinalYearProject\Matrimony_Matchmaker\data\DHS\finalized_descriptive_marriage_data.csv"
processed_df.to_csv(output_path, index=False)

print("Finalized dataframe created with descriptive column names.")
print(processed_df.columns.tolist())

Finalized dataframe created with descriptive column names.
['ID', 'Wife_Age', 'Wife_Education', 'Wife_Profession', 'Wife_Caste', 'Wife_Religion', 'Wife_Residence', 'Husband_Age', 'Husband_Education', 'Husband_Profession', 'Husband_Caste', 'Husband_Religion', 'Husband_Residence', 'Conflict_Score', 'Emotional_Conflict', 'Physical_Conflict', 'Controlling_Negativity', 'Lack_of_Agency', 'Total_Marital_Conflict_Score', 'Marriage_Successful', 'Ever_been_humiliated_by_husband_partner', 'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner', 'Ever_been_pushed_shook_or_had_something_thrown_by_husband_partner', 'Ever_been_slapped_by_husband_partner', 'Husband_partner_jealous_if_respondent_talks_with_other_men', 'Husband_partner_does_not_permit_respondent_to_meet_female_friends', 'Husband_partner_insists_on_knowing_where_respondent_is', 'Person_who_usually_decides_on_respondents_health_care', 'Person_who_usually_decides_on_large_household_purchases', 'Respondent_afraid_of_husband_partner_most

In [21]:
processed_df["Wife_Education"].unique()

array(['Secondary or below', 'Higher Secondary',
       'Graduate (B.Tech/B.Sc/BA)', 'Post-Graduate (MBA/MA)'],
      dtype=object)

In [22]:
processed_df["Husband_Profession"].unique()

array(['Services', 'Sales', 'Agricultural', 'Professional/IT/Tech',
       'Manual', 'Other', 'Clerical', 'Not Working'], dtype=object)

In [23]:
processed_df["Wife_Religion"].unique()

array(['Muslim', 'Hindu', 'Sikh', nan, 'Buddhist', 'Jain', 'Christian'],
      dtype=object)

In [24]:
processed_df["Husband_Religion"].unique()

array(['Muslim', 'Hindu', 'Sikh', 'Christian', nan, 'Buddhist', 'Jain'],
      dtype=object)

In [25]:
processed_df["Husband_Residence"].unique()

array(['Rural', 'Urban'], dtype=object)

In [1]:
import pandas as pd
import re

# 1. Load the raw dataset
file_path = r"D:\FinalYearProject\Matrimony_Matchmaker\data\DHS\IACR7EFL.DTA"
with pd.io.stata.StataReader(file_path) as reader:
    raw_df = reader.read(convert_categoricals=False)

raw_df.columns = [col.lower() for col in raw_df.columns]
# 1. Load raw data (lowercase columns to match text file)
# raw_df = pd.read_stata('IACR7EFL.DTA', convert_categoricals=False)
raw_df.columns = [col.lower() for col in raw_df.columns]

# 2. Read the dictionary text
with open('Couple_data_meaning of coulmns.txt', 'r') as f:
    dict_content = f.read()

# --- PART A: MAP COLUMN HEADERS ---
name_map = {}
header_pattern = re.compile(r'label variable\s+(\w+)\s+"(.+?)"', re.IGNORECASE)
for match in header_pattern.finditer(dict_content):
    clean_desc = match.group(2).replace(" ", "_").replace("/", "_").replace("-", "_")
    name_map[match.group(1).lower()] = clean_desc

# --- PART B: MAP VALUE MEANINGS (THE ANSWERS) ---

# 1. Extract the Answer Keys (label define blocks)
# This captures: label define V025 1 "Urban" 2 "Rural"
value_labels = {}
define_pattern = re.compile(r'label define\s+(\w+)\s*(.*?);', re.DOTALL | re.IGNORECASE)

for match in define_pattern.finditer(dict_content):
    label_name = match.group(1).lower()
    mapping_text = match.group(2)
    
    # Parse individual pairs like: 1 "Urban"
    pairs = re.findall(r'(\d+)\s+"(.+?)"', mapping_text)
    value_labels[label_name] = {int(num): txt for num, txt in pairs}

# 2. Link Columns to Answer Keys (label values lines)
# This captures: label values v025 V025 (Linking column v025 to the Urban/Rural key)
column_to_label_link = {}
link_pattern = re.compile(r'label values\s+(\w+)\s+(\w+)', re.IGNORECASE)

for match in link_pattern.finditer(dict_content):
    col_name = match.group(1).lower()
    label_key = match.group(2).lower()
    column_to_label_link[col_name] = label_key

# 3. Apply the mappings to the dataframe
decoded_df = raw_df.copy()

for col in decoded_df.columns:
    if col in column_to_label_link:
        label_key = column_to_label_link[col]
        if label_key in value_labels:
            # Replace numbers with text using the extracted dictionary
            decoded_df[col] = decoded_df[col].map(value_labels[label_key]).fillna(decoded_df[col])

# --- PART C: ORDERING AND SAVING ---
h_list = ['v730', 'mv438', 'mv133', 'mv717', 'mv131', 'mv130']
w_list = ['v012', 'v438', 'v133', 'v717', 'v131', 'v130', 'v025', 'v024']
c_list = ['d101a', 'd101e', 'd103a', 'd103c', 'd105a', 'd105b', 'v743a', 'v743b', 'd129']

priority_cols = [c for c in (h_list + w_list + c_list) if c in decoded_df.columns]
other_cols = [c for c in decoded_df.columns if c not in priority_cols]

final_df = decoded_df[priority_cols + other_cols].copy()
final_df.rename(columns=name_map, inplace=True)

# Save
final_df.to_csv('Fully_Decoded_India_Dataset.csv', index=False)
print("Headers and Answers decoded for all 3000+ columns.")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11396\1347343322.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  decoded_df[col] = decoded_df[col].map(value_labels[label_key]).fillna(decoded_df[col])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11396\1347343322.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  decoded_df[col] = decoded_df[col].map(value_labels[label_key]).fillna(decoded_df[col])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11396\1347343322.py:58: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill,

Headers and Answers decoded for all 3000+ columns.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("Fully_Decoded_India_Dataset.csv")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4888\3686749774.py:1: DtypeWarning: Columns (6,93,94,95,96,109,110,128,132,157,191,199,260,261,262,263,264,265,266,267,268,269,270,271,340,341,342,343,344,345,346,347,348,349,350,351,360,361,362,363,364,365,366,367,368,369,370,371,376,377,378,379,380,381,382,383,384,385,386,389,440,441,442,443,444,445,446,447,448,449,450,460,461,462,463,464,465,466,467,468,469,470,471,476,478,479,480,497,499,500,501,517,518,519,520,521,522,523,524,525,526,527,528,529,531,539,540,541,542,543,544,545,546,547,548,549,550,558,559,560,561,562,563,564,565,566,567,568,569,570,675,676,694,695,696,713,717,785,791,798,873,880,895,990,1003,1016,1019,1047,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1148,1191,1192,1193,1239,1274,1278,1287,1298,1308,1309,1318,1319,1320,1336,1337,1340,1341,1342,1356,1362,1423,1487,1564,1565,1566,1567,1625,1626,1628,1629,1699,1700,1703,1710,1716,1719,1721,1727,1737,1760,1761,1764,1781,1785,1786,1787,1788,1789,1790,1791,1796,181

In [8]:
# Define the exact column names you want to keep
features = [
    'Husband_age', 
    'Husband years_of_education', 
    'Husband Occupation_(grouped)', 
    'Husband Ethnicity', 
    'Husband Religion', 
    "Wife's_current_age", 
    "Wife's_height_in_centimeters_(1_decimal)", 
    "Wife's Education_in_years", 
    "Wife's_occupation_(grouped)", 
    "Wife's Ethnicity", 
    "Wife's Religion", 
    'residence', 
    'State'
]

targets = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men', 
    'Husband_partner_insists_on_knowing_where_Wife_is', 
    'Ever_been_humiliated_by_husband_partner', 
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner', 
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner', 
    'Ever_been_slapped_by_husband_partner', 
    "Person_who_usually_decides_on_Wife's_health_care", 
    "Person_who_usually_decides_on_large_household_purchases", 
    "Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never"
]

# Create the new DataFrame with only these columns
# We use intersection() to avoid errors if a column name has a slight typo
cols_to_keep = [c for c in (features + targets) if c in df.columns]
final_df = df[cols_to_keep].copy()

# View the result
print(f"New DataFrame shape: {final_df.shape}")
print(final_df.head())

New DataFrame shape: (57693, 22)
   Husband_age  Husband years_of_education       Husband Occupation_(grouped)  \
0           31                          12  Services / household and domestic   
1           42                           5                              Sales   
2           54                           8                       Agricultural   
3           38                           7                       Agricultural   
4           39                          12                       Agricultural   

  Husband Ethnicity Husband Religion  Wife's_current_age  \
0             Caste           Muslim                  29   
1             Caste           Muslim                  36   
2             Tribe           Muslim                  48   
3             Caste           Muslim                  39   
4             Tribe           Muslim                  31   

  Wife's_height_in_centimeters_(1_decimal)  Wife's Education_in_years  \
0                                     1617    

In [ ]:
# Helper function to map education years to your specific B.Tech/MBA format
def map_degree(years):
    if pd.isna(years) or years > 90: return "N/A"
    if years >= 17: return "Post-Graduate (MBA/MA)"
    if years >= 15: return "Graduate (B.Tech/B.Sc/BA)"
    if years >= 12: return "Higher Secondary"
    return "Secondary or below"
final_df["Husband years_of_education"] = final_df["Husband years_of_education"].apply(map_degree)
final_df["Wife's Education_in_years"] = final_df["Wife's Education_in_years"].apply(map_degree)
final_df.rename(columns={"Husband years_of_education" : "Husband's Education Level"}, inplace=True)
final_df.rename(columns={"Wife's Education_in_years" : "Wife's Education Level"}, inplace=True)
final_df.rename(columns={"Wife's_height_in_centimeters_(1_decimal)" : "Wife's height(centimeters)"}, inplace=True)



In [19]:
final_df["Wife's height(centimeters)"] = (
    pd.to_numeric(final_df["Wife's height(centimeters)"], errors="coerce") / 10
).round(2)


In [21]:
final_df.head()
final_df.to_csv("dataset_final_form.csv", index=False)

All Survey Questions Values

A. Person who usually decides on respondent's (Wife's) health care.
    1: Respondent alone (Wife decides)

    2: Husband/partner alone (Husband decides)

    3: Respondent and husband/partner jointly (Joint Decision - Ideal)

    4: Someone else

    5: Respondent and someone else

    6: Decision not made/not applicable

B.Person who usually decides on large household purchases
    1: Respondent alone

    2: Husband/partner alone

    3: Respondent and husband/partner jointly (Joint Decision - Ideal)

    4: Someone else

    5: Respondent and someone else

    6: Decision not made/not applicable

C.Husband/partner jealous if wife talks with other men.
    0: No

    1: Yes
    
    8: Don't know

D.Husband/partner insists on knowing where wife is at all times.

    0: No
    
    1: Yes
    
    8: Don't know

E.Ever been humiliated by husband/partner.

    0: Never

    1: Often

    2: Sometimes

    3: Yes, but not in the last 12 months

    4: Yes, but frequency missing

F.Ever been insulted or made to feel bad by husband/partner.

    0: Never

    1: Often

    2: Sometimes

    3: Yes, but not in the last 12 months

    4: Yes, but frequency missing

G.Ever been pushed, shook, or had something thrown by husband/partner.

    0: Never
    
    1: Often
    
    2: Sometimes
    
    3: Yes, but not in the last 12 months
    
    4: Yes, but frequency missing

H.Ever been slapped by husband/partner.

    0: Never

    1: Often

    2: Sometimes

    3: Yes, but not in the last 12 months

    4: Yes, but frequency missing

I.Wife afraid of husband/partner most of the time, sometimes, or never.

    0: Never
    
    1: Sometimes
    
    2: Most of the time

Based on the "Predicting Marital Stability" (Frank, 2024) paper, 
the "Relationship Compatibility" dimension was built on 
6 specific psychological questions.

In [10]:
df = pd.read_csv(r"/workspaces/FinalYearProject/Matrimony_Matchmaker/data/DHS_DATA.csv")
print(df.columns)

Index(['Husband_age', 'Husband's Education Level',
       'Husband Occupation_(grouped)', 'Husband Ethnicity', 'Husband Religion',
       'Wife's_current_age', 'Wife's height(centimeters)',
       'Wife's Education Level', 'Wife's_occupation_(grouped)',
       'Wife's Ethnicity', 'Wife's Religion', 'residence', 'State',
       'Husband_partner_jealous_if_Wife_talks_with_other_men',
       'Husband_partner_insists_on_knowing_where_Wife_is',
       'Ever_been_humiliated_by_husband_partner',
       'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
       'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
       'Ever_been_slapped_by_husband_partner',
       'Person_who_usually_decides_on_Wife's_health_care',
       'Person_who_usually_decides_on_large_household_purchases',
       'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'],
      dtype='object')


In [11]:
# 1. Load your dataset (Replace 'your_file.csv' with your actual filename)
# df = pd.read_csv('your_file.csv') 

# 2. Define the 9 Target Columns (Exact names from your list)
target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

# 3. Check for "Missing" Values
# Note: In some datasets, "missing" might be represented as NaN, or blank strings, 
# or specific codes like "Missing", "Don't know".
# We assume standard NaNs here. If you have "Don't know" as a string, add it to the replace list.

# Optional: Convert "Don't know" or "Missing" strings to NaN for accurate filtering
# df[target_cols] = df[target_cols].replace(['Missing', 'Don\'t know', '.'], np.nan)

# 4. Create the Masks
# Mask for Complete Rows: All 9 columns must have a value (not NaN)
mask_complete = df[target_cols].notna().all(axis=1)

# Mask for Incomplete Rows: At least one column is NaN
mask_incomplete = ~mask_complete

# 5. Split the Data
df_complete = df[mask_complete].copy()
df_incomplete = df[mask_incomplete].copy()

# 6. Verify Counts
print(f"Total Rows: {len(df)}")
print(f"Complete Responses (To Keep): {len(df_complete)}")
print(f"Incomplete Responses (To Separate): {len(df_incomplete)}")

# 7. Save to Files
df_complete.to_csv('completeresponse_dhs.csv', index=False)
df_incomplete.to_csv('incompleteresponse_dhs.csv', index=False)

print("\nFiles saved successfully: 'completeresponse_dhs.csv' and 'incompleteresponse_dhs.csv'")

Total Rows: 57693
Complete Responses (To Keep): 46488
Incomplete Responses (To Separate): 11205

Files saved successfully: 'completeresponse_dhs.csv' and 'incompleteresponse_dhs.csv'


The exact Scoring Dictionary to map text-based values to the 0–4 Risk Scale used in "Predicting Marital Stability: An Approach for More Characteristics (Frank, 2024)".

The Logic:

    0: Perfect Compatibility (Safe, Joint Decisions, No Fear).

    4: High Incompatibility (Violence, Total Control, High Fear).

1. ROLES (Decision Making)
-Person_who_usually_decides_on_Wife's_health_careConcept: Egalitarian vs. Hierarchical roles.
Text Value to Score:
    "Respondent and husband/partner jointly" - > 0 (Ideal/Compatible)
    "Respondent alone" -> 1 (Autonomous/Safe)
    "Husband/partner alone" -> 4 (Hierarchical/Risk)
    "Someone else" / "Other" -> 4 (Lack of Couple Autonomy)
----------------------------------------------------------------

2. GOALS (Financial Shared Vision) - Person_who_usually_decides_on_large_household_purchases
Text Value to Score: 
    "Respondent and husband/partner jointly" -> 0 (Shared Goals)
    "Respondent alone" -> 1 (Safe)
    "Husband/partner alone" -> 4 (Mismatched Goals/Control)
-----------------------------------------------------------------

3. TRUST (Jealousy & Control) - 
Husband_partner_jealous_if_Wife_talks_with_other_men + 
Husband_partner_insists_on_knowing_where_Wife_is
Text Value to Score:
    "No" ->0 (High Trust)
    "Yes" -> 4 (Low Trust / High Risk)
    "Don't know" -> 2 (Uncertainty)
-----------------------------------------------------------------

4. VIEWS ON MARRIAGE (Boundaries/Violence) -
Ever_been_pushed,_shook_or_had_something_thrown + 
Ever_been_slapped_by_husband_partner
    "Never" -> 0 (Compatible Views)
    "Yes, but not in the last 12 months" -> 2 (Past Issues/Moderate Risk)
    "Sometimes" -> 3 (Ongoing Incompatibility)
    "Often" -> 4 (Severe Incompatibility)
-----------------------------------------------------------------

5. LIFE GOALS (Fear)
Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never
    "Never" -> 0 (Aligned)
    "Sometimes" -> 2 (Friction)
    "Most of the time" -> 4 (Divergent/Toxic)
-----------------------------------------------------------------

6. DREAMS (Emotional Support vs. Abuse)
Ever_been_humiliated_by_husband_partner + 
Ever_been_insulted_or_made_to_feel_bad...
    "Never" -> 0 (Supportive)
    "Yes, but not in the last 12 months" -> 2 (Past Risk)"Sometimes" -> 3 (Damaging)
    "Often" -> 4 (Dream-Killing/High Risk)
-----------------------------------------------------------------

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('completeresponse_dhs.csv')
# Load your Complete Data
# df = pd.read_csv('completeresponse_dhs.csv')

# --- 1. DEFINE SCALING FUNCTIONS ---

def score_decisions(val):
    s = str(val).lower()
    if 'joint' in s: return 0        # Best
    if 'respondent' in s: return 1   # Good
    if 'husband' in s: return 4      # Bad
    if 'someone else' in s: return 4 # Bad
    return 2 # Neutral/Unknown

def score_binary_risk(val):
    # For Jealousy columns
    s = str(val).lower()
    if 'no' in s: return 0
    if 'yes' in s: return 4
    return 2

def score_frequency_risk(val):
    # For Violence/Humiliation columns
    s = str(val).lower()
    if 'never' in s: return 0
    if 'not in the last' in s: return 2
    if 'sometimes' in s: return 3
    if 'often' in s: return 4
    return 2

def score_fear(val):
    s = str(val).lower()
    if 'never' in s: return 0
    if 'sometimes' in s: return 2
    if 'most' in s: return 4
    return 0

# --- 2. APPLY MAPPING TO CREATE THE 6 DIMENSIONS ---

# Dim 1: Roles
df['Score_Roles'] = df["Person_who_usually_decides_on_Wife's_health_care"].apply(score_decisions)

# Dim 2: Goals
df['Score_Goals'] = df["Person_who_usually_decides_on_large_household_purchases"].apply(score_decisions)

# Dim 3: Trust (Average of Jealousy + Control)
t1 = df['Husband_partner_jealous_if_Wife_talks_with_other_men'].apply(score_binary_risk)
t2 = df['Husband_partner_insists_on_knowing_where_Wife_is'].apply(score_binary_risk)
df['Score_Trust'] = (t1 + t2) / 2

# Dim 4: Views (Average of Push + Slap)
v1 = df['Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner'].apply(score_frequency_risk)
v2 = df['Ever_been_slapped_by_husband_partner'].apply(score_frequency_risk)
df['Score_Views'] = (v1 + v2) / 2

# Dim 5: Life Goals (Fear)
df['Score_LifeGoals'] = df["Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never"].apply(score_fear)

# Dim 6: Dreams (Average of Humiliation + Insult)
d1 = df['Ever_been_humiliated_by_husband_partner'].apply(score_frequency_risk)
d2 = df['Ever_been_insulted_or_made_to_feel_bad_by_husband_partner'].apply(score_frequency_risk)
df['Score_Dreams'] = (d1 + d2) / 2

# --- 3. FINAL AGGREGATION ---
# Average of the 6 dimensions
df['Frank_Compatibility_Score'] = df[['Score_Roles', 'Score_Goals', 'Score_Trust', 
                                      'Score_Views', 'Score_LifeGoals', 'Score_Dreams']].mean(axis=1)

# --- 4. DETERMINE TARGET CLASS ---
# Threshold = 1.227 (from Paper)
# <= 1.227 : Compatible (Class 1)
# > 1.227  : Incompatible (Class 0)
df['Is_Compatible'] = np.where(df['Frank_Compatibility_Score'] <= 1.227, 1, 0)

# Save
df.to_csv('Target_Variable_Created.csv', index=False)
print(df[['Frank_Compatibility_Score', 'Is_Compatible']].head(10))

   Frank_Compatibility_Score  Is_Compatible
0                   0.666667              1
1                   1.000000              1
2                   0.333333              1
3                   0.666667              1
4                   1.333333              0
5                   0.333333              1
6                   0.333333              1
7                   0.333333              1
8                   0.333333              1
9                   0.333333              1


In [ ]:
df['Is_Compatible'].value_counts()

Is_Compatible
1    31108
0    15380
Name: count, dtype: int64

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_curve

# --- 1. LOAD & PREPARE DATA ---
df_features = pd.read_csv('completeresponse_dhs.csv')
df_target = pd.read_csv('Target_Variable_Created.csv')
df = pd.concat([df_features, df_target[['Is_Compatible']]], axis=1)

feature_cols = [
    'Husband_age', "Husband's Education Level", 'Husband Occupation_(grouped)', 
    'Husband Religion', 'Husband Ethnicity',
    "Wife's_current_age", "Wife's Education Level", "Wife's_occupation_(grouped)", 
    "Wife's Religion", "Wife's Ethnicity"
]

X = pd.get_dummies(df[feature_cols], drop_first=True)
y = df['Is_Compatible']

# --- 2. TRAIN/TEST SPLIT ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 3. AGGRESSIVE MODEL TRAINING ---
# We deliberately OVER-weight the minority class to force the model to learn it.
# Standard ratio was ~2. We are pumping it to 10.
aggressive_weight = 10 

print(f"\nTraining with Aggressive Weight: {aggressive_weight}")
model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.03,    # Slower learning to find subtle patterns
    max_depth=6,           
    scale_pos_weight=aggressive_weight, # FORCE FOCUS ON CLASS 0
    subsample=0.8,         # Prevent overfitting
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train, y_train)

# --- 4. THRESHOLD TUNING (The Magic Step) ---
# Instead of predict(), we get probabilities (0.0 to 1.0)
y_proba = model.predict_proba(X_test)[:, 1] # Probability of being Compatible (Class 1)

# We want to catch Class 0 (Incompatible).
# Standard logic: If Prob(Compatible) < 0.5, then Incompatible.
# NEW logic: We will be stricter. We only call it "Compatible" if Prob > 0.7.
# This makes it much easier to fall into the "Incompatible" bucket.

decision_threshold = 0.65  # <--- ADJUST THIS TO BALANCE RESULTS
y_pred_tuned = (y_proba > decision_threshold).astype(int)

# --- 5. EVALUATE ---
print(f"\n--- Performance at Threshold {decision_threshold} ---")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_tuned))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_tuned))

# --- 6. AUTOMATIC OPTIMAL THRESHOLD FINDER ---
# This loop finds the threshold that gives the best 'Macro F1' score
print("\n--- Searching for Best Threshold ---")
best_thresh = 0
best_score = 0
for thresh in np.arange(0.3, 0.9, 0.05):
    preds = (y_proba > thresh).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
    # Focus on Class 0 Recall (Catching Violence)
    recall_0 = tn / (tn + fp) if (tn + fp) > 0 else 0
    # But don't destroy Precision
    precision_0 = tn / (tn + fn) if (tn + fn) > 0 else 0
    
    f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0) if (precision_0 + recall_0) > 0 else 0
    
    if f1_0 > best_score:
        best_score = f1_0
        best_thresh = thresh

print(f"Optimal Threshold for Class 0 Detection: {best_thresh:.2f}")
print(f"Best F1-Score for Class 0: {best_score:.3f}")


Training with Aggressive Weight: 10


/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:52:37] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- Performance at Threshold 0.65 ---
Confusion Matrix:
[[   2 3074]
 [   2 6220]]

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.00      0.00      3076
           1       0.67      1.00      0.80      6222

    accuracy                           0.67      9298
   macro avg       0.58      0.50      0.40      9298
weighted avg       0.61      0.67      0.54      9298


--- Searching for Best Threshold ---
Optimal Threshold for Class 0 Detection: 0.90
Best F1-Score for Class 0: 0.015


In [1]:
import pandas as pd
df_features = pd.read_csv('completeresponse_dhs.csv')


XGBoost Model is used to predict the ans to all 9 Questions. Since, mapping 9 to 1 marital satisfaction gave poor accuracy and recall.
MultiOutPut variant of XGBoost is used.(All 9 questions are treated independently and no correlation among them is consdered)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

feature = [c for c in df_features.columns if c not in target_cols]
X = df_features[feature].copy()
y = df_features[target_cols].copy()

# 6. Encode Categorical Features
X_encoded = pd.get_dummies(X, drop_first=True)

# 7. Encode the 9 Target Variables
label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    # XGBoost requires targets to be integer encoded (0, 1, 2...)
    y[col] = le.fit_transform(y[col].astype(str))
    label_encoders[col] = le

# 8. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# 9. Build and Train the Multi-Output XGBoost Model
xgb = XGBClassifier(eval_metric='mlogloss', random_state=42)
multi_target_xgb = MultiOutputClassifier(xgb, n_jobs=-1)

print("Training multi-output XGBoost model...")
multi_target_xgb.fit(X_train, y_train)

# 10. Predict and Evaluate
y_pred_test = multi_target_xgb.predict(X_test)

print("\n--- Test Accuracy per Survey Question ---")
test_accuracies = []
for i, col in enumerate(target_cols):
    acc = accuracy_score(y_test.iloc[:, i], y_pred_test[:, i])
    test_accuracies.append(acc)
    print(f"{col}: {acc * 100:.2f}%")

print(f"\nAverage Overall Test Accuracy: {np.mean(test_accuracies) * 100:.2f}%")

Training multi-output XGBoost model...

--- Test Accuracy per Survey Question ---
Husband_partner_jealous_if_Wife_talks_with_other_men: 74.35%
Husband_partner_insists_on_knowing_where_Wife_is: 80.60%
Ever_been_humiliated_by_husband_partner: 91.85%
Ever_been_insulted_or_made_to_feel_bad_by_husband_partner: 92.77%
Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner: 89.17%
Ever_been_slapped_by_husband_partner: 76.44%
Person_who_usually_decides_on_Wife's_health_care: 73.60%
Person_who_usually_decides_on_large_household_purchases: 73.82%
Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never: 65.82%

Average Overall Test Accuracy: 79.82%


ClassiferChain variant(of XGBoost) however, sees the effect of one already predicted question on the other. Eg: 
if a husband has slapped his wife, the probability that she is afraid of him goes up significantly.

In [7]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import ClassifierChain
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)

# 1. Load your dataset (Replace with your actual dataframe if it's already loaded)
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

# Ensure no targets are missing
df_features = df_features.dropna(subset=target_cols).copy()

feature_cols = [c for c in df_features.columns if c not in target_cols]
X = df_features[feature_cols].copy()
y = df_features[target_cols].copy()

# ---------------------------------------------------------
# Robust Missing Value Handling
# ---------------------------------------------------------
# Fixed the dtype warning by adding 'str' 
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

# Impute numeric columns with median 
num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

# Impute categorical columns with a placeholder 'Unknown'
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
# ---------------------------------------------------------

# Encode Categorical Features
X_encoded = pd.get_dummies(X, drop_first=True)

# Encode the 9 Target Variables
label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    y[col] = le.fit_transform(y[col].astype(str))
    label_encoders[col] = le

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Build and Train the ClassifierChain XGBoost Model
xgb = XGBClassifier(
    learning_rate=0.05,
    n_estimators=150,
    max_depth=5,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss', 
    random_state=42
)

chain_xgb = ClassifierChain(xgb, order='random', random_state=42)

print("Training ClassifierChain XGBoost model...")
chain_xgb.fit(X_train, y_train)

# Predict and Evaluate
y_pred_test = chain_xgb.predict(X_test)

print("\n--- Test Results per Survey Question ---")
test_accuracies = []

# We use enumerate(target_cols) to get the index 'i' to match the columns in y_pred_test
for i, col in enumerate(target_cols):
    
    # Calculate Overall Accuracy for this column
    acc = accuracy_score(y_test.iloc[:, i], y_pred_test[:, i])
    test_accuracies.append(acc)
    
    print(f"\n==================================================")
    print(f"=== Report for: {col} ===")
    print(f"Overall Accuracy: {acc * 100:.2f}%")
    print(f"--------------------------------------------------")
    
    # We use the label encoder to get the actual text labels (e.g., 'Never', 'Yes')
    actual_labels = label_encoders[col].classes_
    
    # Print Precision, Recall, and F1 for EACH specific answer
    # Adding zero_division=0 prevents annoying red warnings for rare classes
    print(classification_report(
        y_test.iloc[:, i], 
        y_pred_test[:, i], 
        target_names=actual_labels,
        zero_division=0
    ))

print(f"\n======================================")
print(f"Average Overall Test Accuracy: {np.mean(test_accuracies) * 100:.2f}%")
print(f"======================================")

Training ClassifierChain XGBoost model...

--- Test Results per Survey Question ---

=== Report for: Husband_partner_jealous_if_Wife_talks_with_other_men ===
Overall Accuracy: 74.94%
--------------------------------------------------
              precision    recall  f1-score   support

  Don't know       0.00      0.00      0.00        31
          No       0.75      1.00      0.86      6968
         Yes       0.00      0.00      0.00      2299

    accuracy                           0.75      9298
   macro avg       0.25      0.33      0.29      9298
weighted avg       0.56      0.75      0.64      9298


=== Report for: Husband_partner_insists_on_knowing_where_Wife_is ===
Overall Accuracy: 80.66%
--------------------------------------------------
              precision    recall  f1-score   support

  Don't know       0.00      0.00      0.00        12
          No       0.81      1.00      0.89      7501
         Yes       0.00      0.00      0.00      1785

    accuracy         

UPDATED CLASSIFIER CHAIN KEEPING IN MIND THE IMBALANCE IN DATA.



In [9]:
# --- PRINT FREQUENCIES FOR THE REVIEW BOARD ---
print("==================================================")
print("DATASET IMBALANCE (FREQUENCY PER QUESTION)")
print("==================================================\n")
for col in target_cols:
    print(df_features[col].value_counts())
    print("\n")

DATASET IMBALANCE (FREQUENCY PER QUESTION)

Husband_partner_jealous_if_Wife_talks_with_other_men
No            34834
Yes           11498
Don't know      156
Name: count, dtype: int64


Husband_partner_insists_on_knowing_where_Wife_is
No            37603
Yes            8821
Don't know       64
Name: count, dtype: int64


Ever_been_humiliated_by_husband_partner
Never                                 42919
Sometimes                              2540
Often                                   592
Yes, but not in the last 12 months      437
Name: count, dtype: int64


Ever_been_insulted_or_made_to_feel_bad_by_husband_partner
Never                                 43320
Sometimes                              2258
Often                                   523
Yes, but not in the last 12 months      387
Name: count, dtype: int64


Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner
Never                                 41446
Sometimes                              3576
Yes, but not in t

In [10]:
# --- PRINT FREQUENCIES FOR THE REVIEW BOARD ---
print("==================================================")
print("DATASET IMBALANCE (FREQUENCY PER QUESTION)")
print("==================================================\n")
for col in target_cols:
    print(y_train[col].value_counts())
    print("\n")

DATASET IMBALANCE (FREQUENCY PER QUESTION)

Husband_partner_jealous_if_Wife_talks_with_other_men
1    27866
2     9199
0      125
Name: count, dtype: int64


Husband_partner_insists_on_knowing_where_Wife_is
1    30102
2     7036
0       52
Name: count, dtype: int64


Ever_been_humiliated_by_husband_partner
0    34379
2     2000
1      469
3      342
Name: count, dtype: int64


Ever_been_insulted_or_made_to_feel_bad_by_husband_partner
0    34691
2     1780
1      409
3      310
Name: count, dtype: int64


Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner
0    33150
2     2865
3      729
1      446
Name: count, dtype: int64


Ever_been_slapped_by_husband_partner
0    28554
2     6035
3     1919
1      682
Name: count, dtype: int64


Person_who_usually_decides_on_Wife's_health_care
3    27260
0     6346
2     3153
4      278
1      153
Name: count, dtype: int64


Person_who_usually_decides_on_large_household_purchases
3    27542
0     6377
2     2317
4      708
1      246

In [11]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_sample_weight

warnings.filterwarnings('ignore', category=FutureWarning)

# 1. Load Data
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()


# Prepare Features and Targets
feature_cols = [c for c in df_features.columns if c not in target_cols]
X = df_features[feature_cols].copy()
y = df_features[target_cols].copy()

# Robust Missing Value Handling
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

X_encoded = pd.get_dummies(X, drop_first=True)

label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    y[col] = le.fit_transform(y[col].astype(str))
    label_encoders[col] = le

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# --- TRAIN INDEPENDENT, NATIVELY OPTIMIZED MODELS ---
print("==================================================")
print("TRAINING INDEPENDENT XGBOOST MODELS")
print("==================================================\n")

test_accuracies = []

for col in target_cols:
    
    # Compute balanced weights for this specific question
    weights = compute_sample_weight(class_weight='balanced', y=y_train[col])
    
    # XGBoost optimized for extreme imbalance without synthetic data
    xgb = XGBClassifier(
        learning_rate=0.05,
        n_estimators=200,
        max_depth=5,
        max_delta_step=5,        # <-- CRITICAL: Prevents 0% recall on rare classes
        min_child_weight=1,      # Allow learning from very few minority samples
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric='mlogloss', 
        random_state=42,
        n_jobs=-1
    )
    
    # Pass the mathematical weights into the fitting process
    xgb.fit(X_train, y_train[col], sample_weight=weights)
    
    y_pred_test = xgb.predict(X_test)
    
    acc = accuracy_score(y_test[col], y_pred_test)
    test_accuracies.append(acc)
    
    print(f"\n==================================================")
    print(f"=== Report for: {col} ===")
    print(f"Overall Accuracy: {acc * 100:.2f}%")
    print(f"--------------------------------------------------")
    
    actual_labels = label_encoders[col].classes_
    
    print(classification_report(
        y_test[col], 
        y_pred_test, 
        target_names=actual_labels,
        zero_division=0
    ))

print(f"\n======================================")
print(f"Average Overall Test Accuracy: {np.mean(test_accuracies) * 100:.2f}%")
print(f"======================================")

TRAINING INDEPENDENT XGBOOST MODELS


=== Report for: Husband_partner_jealous_if_Wife_talks_with_other_men ===
Overall Accuracy: 55.99%
--------------------------------------------------
              precision    recall  f1-score   support

  Don't know       0.01      0.23      0.02        31
          No       0.81      0.56      0.67      6968
         Yes       0.33      0.55      0.42      2299

    accuracy                           0.56      9298
   macro avg       0.39      0.45      0.37      9298
weighted avg       0.69      0.56      0.60      9298


=== Report for: Husband_partner_insists_on_knowing_where_Wife_is ===
Overall Accuracy: 57.13%
--------------------------------------------------
              precision    recall  f1-score   support

  Don't know       0.00      0.08      0.01        12
          No       0.85      0.58      0.69      7501
         Yes       0.25      0.54      0.34      1785

    accuracy                           0.57      9298
   macro avg  

In [12]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

# 1. Load Data
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()

feature_cols = [c for c in df_features.columns if c not in target_cols]
X = df_features[feature_cols].copy()
y = df_features[target_cols].copy()

# Robust Missing Value Handling
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

X_encoded = pd.get_dummies(X, drop_first=True)

label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    y[col] = le.fit_transform(y[col].astype(str))
    label_encoders[col] = le

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Define the Critical Red Flag classes we want to optimize for
red_flags = ['Yes', 'Often', 'Sometimes', 'Husband/partner alone', 'Most of the time afraid']

print("==================================================")
print("TRAINING PURE MODELS & OPTIMIZING DECISION THRESHOLDS")
print("==================================================\n")

test_accuracies = []

for col in target_cols:
    # 1. Train a 100% PURE model (No weights, no SMOTE)
    xgb = XGBClassifier(
        learning_rate=0.05,
        n_estimators=150,
        max_depth=6,
        eval_metric='mlogloss', 
        random_state=42,
        n_jobs=-1
    )
    xgb.fit(X_train, y_train[col])
    
    # 2. Extract RAW Probabilities
    y_probs = xgb.predict_proba(X_test)
    classes = label_encoders[col].classes_
    
    # Identify which indices belong to the red flags
    red_flag_indices = [idx for idx, c in enumerate(classes) if c in red_flags]
    
    # 3. Find the threshold that maximizes the F1-Score for Red Flags
    best_threshold = 0.50 # Default XGBoost threshold
    best_f1 = -1
    best_y_pred = []
    
    # We test thresholds from 10% to 50%
    thresholds = np.arange(0.10, 0.51, 0.02) 
    
    if red_flag_indices:
        for thresh in thresholds:
            y_pred_temp = []
            for prob_row in y_probs:
                best_class = np.argmax(prob_row) # Default prediction
                # Override if a red flag class hits the custom threshold
                for rf_idx in red_flag_indices:
                    if prob_row[rf_idx] >= thresh:
                        best_class = rf_idx
                        break
                y_pred_temp.append(best_class)
            
            # Calculate F1 score for the red flag classes present in test data
            labels_present = np.unique(y_test[col])
            rf_present = [idx for idx in red_flag_indices if idx in labels_present]
            
            if rf_present:
                current_f1 = f1_score(y_test[col], y_pred_temp, labels=rf_present, average='macro', zero_division=0)
                if current_f1 > best_f1:
                    best_f1 = current_f1
                    best_threshold = thresh
                    best_y_pred = y_pred_temp
    else:
        # If no red flags in this question, just use default predictions
        best_y_pred = np.argmax(y_probs, axis=1)
    
    # Evaluate the optimized predictions
    acc = accuracy_score(y_test[col], best_y_pred)
    test_accuracies.append(acc)
    
    print(f"\n==================================================")
    print(f"=== Report for: {col} ===")
    if red_flag_indices:
        print(f"Optimized Decision Threshold: {best_threshold*100:.0f}% (Maximizes F1-Score)")
    print(f"Overall Accuracy: {acc * 100:.2f}%")
    print(f"--------------------------------------------------")
    
    print(classification_report(
        y_test[col], 
        best_y_pred, 
        target_names=classes,
        zero_division=0
    ))

print(f"\n======================================")
print(f"Average Overall Test Accuracy: {np.mean(test_accuracies) * 100:.2f}%")
print(f"======================================")

TRAINING PURE MODELS & OPTIMIZING DECISION THRESHOLDS


=== Report for: Husband_partner_jealous_if_Wife_talks_with_other_men ===
Optimized Decision Threshold: 20% (Maximizes F1-Score)
Overall Accuracy: 48.34%
--------------------------------------------------
              precision    recall  f1-score   support

  Don't know       0.00      0.00      0.00        31
          No       0.85      0.38      0.53      6968
         Yes       0.30      0.79      0.43      2299

    accuracy                           0.48      9298
   macro avg       0.38      0.39      0.32      9298
weighted avg       0.71      0.48      0.50      9298


=== Report for: Husband_partner_insists_on_knowing_where_Wife_is ===
Optimized Decision Threshold: 16% (Maximizes F1-Score)
Overall Accuracy: 45.73%
--------------------------------------------------
              precision    recall  f1-score   support

  Don't know       0.00      0.00      0.00        12
          No       0.87      0.39      0.53      

Method 2: Unsupervised Clustering (Data-Driven)
This is exactly what the research paper did. Instead of manually guessing the weights, we use an unsupervised machine learning algorithm (K-Means) on the 9 survey questions.

The algorithm will mathematically group the couples into distinct clusters—for example, Cluster 0 ("Stable/Low Risk") and Cluster 1 ("High Friction/Abuse"). This becomes our single, clean target for XGBoost.

In [13]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

# 1. Load Data
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()

# --- STEP A: ENCODE THE 9 QUESTIONS FOR CLUSTERING ---
y_raw = df_features[target_cols].copy()
label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    y_raw[col] = le.fit_transform(y_raw[col].astype(str))
    label_encoders[col] = le

# --- STEP B: K-MEANS CLUSTERING (From the Research Paper) ---
print("Running Clustering Analysis to create a Single Target...")
# We ask the algorithm to find 2 distinct types of marriages based on the 9 answers
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_features['Marital_Stability_Cluster'] = kmeans.fit_predict(y_raw)

# Now, 'Marital_Stability_Cluster' is our SINGLE target (0 or 1)
single_target = 'Marital_Stability_Cluster'

print(f"\nCluster Distribution:")
print(df_features[single_target].value_counts())

# --- STEP C: PREPARE FEATURES ---
feature_cols = [c for c in df_features.columns if c not in target_cols and c != single_target]
X = df_features[feature_cols].copy()
y = df_features[single_target].copy()

# Robust Missing Value Handling
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

X_encoded = pd.get_dummies(X, drop_first=True)

# --- STEP D: TRAIN XGBOOST ON THE SINGLE TARGET ---
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

xgb = XGBClassifier(
    learning_rate=0.05,
    n_estimators=200,
    max_depth=6,
    eval_metric='logloss', 
    random_state=42,
    n_jobs=-1
)

print("\nTraining XGBoost on the Single Clustered Target...")
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

print(f"\n==================================================")
print(f"=== FINAL SINGLE TARGET MODEL PERFORMANCE ===")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(f"--------------------------------------------------")
print(classification_report(y_test, y_pred))

Running Clustering Analysis to create a Single Target...

Cluster Distribution:
Marital_Stability_Cluster
0    35844
1    10644
Name: count, dtype: int64

Training XGBoost on the Single Clustered Target...

=== FINAL SINGLE TARGET MODEL PERFORMANCE ===
Overall Accuracy: 77.37%
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.78      0.99      0.87      7206
           1       0.41      0.01      0.03      2092

    accuracy                           0.77      9298
   macro avg       0.59      0.50      0.45      9298
weighted avg       0.69      0.77      0.68      9298



Because we stripped out all the imbalance handling to run a clean clustering baseline, the XGBoost algorithm looked at the 77% (Class 0) to 23% (Class 1) split and realized that simply guessing "0" for almost everyone would easily secure a 77% overall accuracy. It completely ignored the minority class to farm easy accuracy points.

Fix: 

scale_pos_weight + Binary Thresholding

This parameter is designed specifically for binary targets. It mathematically calculates the exact ratio of the majority class to the minority class (e.g., 35,844 / 10,644 = ~3.36) and multiplies the penalty for missing a Class 1 by that exact amount.

optimized by F1-score

In [14]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

# 1. Load Data (assuming df_features is ready)
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()

# --- ENCODE AND CLUSTER ---
y_raw = df_features[target_cols].copy()
label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    y_raw[col] = le.fit_transform(y_raw[col].astype(str))

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_features['Marital_Stability_Cluster'] = kmeans.fit_predict(y_raw)
single_target = 'Marital_Stability_Cluster'

# Check which cluster is the minority (the 'red flag' cluster)
value_counts = df_features[single_target].value_counts()
minority_class = value_counts.idxmin()
majority_class = value_counts.idxmax()

# --- PREPARE FEATURES ---
feature_cols = [c for c in df_features.columns if c not in target_cols and c != single_target]
X = df_features[feature_cols].copy()
y = df_features[single_target].copy()

# Robust Missing Value Handling
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

X_encoded = pd.get_dummies(X, drop_first=True)

# --- TRAIN-TEST SPLIT ---
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Calculate exact weight ratio for the minority class
ratio = float(np.sum(y_train == majority_class)) / np.sum(y_train == minority_class)
print(f"Calculated scale_pos_weight: {ratio:.2f}")

# --- TRAIN WEIGHTED BINARY XGBOOST ---
xgb = XGBClassifier(
    learning_rate=0.05,
    n_estimators=200,
    max_depth=6,
    scale_pos_weight=ratio,  # <-- THE FIX: Forces model to respect the minority class
    eval_metric='logloss', 
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)

# --- THRESHOLD OPTIMIZATION ---
y_probs = xgb.predict_proba(X_test)[:, minority_class] # Get probabilities specifically for the minority class

best_threshold = 0.50
best_f1 = -1
best_y_pred = []

# Sweep thresholds to find the perfect FP/FN balance
for thresh in np.arange(0.10, 0.90, 0.02):
    # If probability of minority class > threshold, predict minority, else majority
    y_pred_temp = np.where(y_probs >= thresh, minority_class, majority_class)
    current_f1 = f1_score(y_test, y_pred_temp, pos_label=minority_class)
    
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_threshold = thresh
        best_y_pred = y_pred_temp

print(f"\n==================================================")
print(f"=== OPTIMIZED BINARY CLUSTER PERFORMANCE ===")
print(f"Minority Class (High Risk): {minority_class}")
print(f"Optimal Decision Threshold: {best_threshold*100:.0f}%")
print(f"Overall Accuracy: {accuracy_score(y_test, best_y_pred) * 100:.2f}%")
print(f"--------------------------------------------------")
print(classification_report(y_test, best_y_pred))

Calculated scale_pos_weight: 3.35

=== OPTIMIZED BINARY CLUSTER PERFORMANCE ===
Minority Class (High Risk): 1
Optimal Decision Threshold: 44%
Overall Accuracy: 44.13%
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.84      0.35      0.49      7206
           1       0.26      0.77      0.38      2092

    accuracy                           0.44      9298
   macro avg       0.55      0.56      0.44      9298
weighted avg       0.71      0.44      0.47      9298



1. Age Heterogamy (The Age Gap)

High Age Difference Flag: The literature highlights that an age difference of 7 or more years between spouses is a significant predictor of marital breakdown.


Wife Older Flag: Marriages where the wife is older than her husband (e.g., the wife marries at 22 or older and the husband is 21 or younger) are subject to significantly higher dissolution rates due to unorthodox social stressors.
+1


Extreme Age Gap Flag: A specific heterogamous stressor occurs when a woman marries as a teenager while her husband is significantly older (e.g., 28 or older).

2. Educational Heterogamy (Education Disparity)

Spousal Education Differential: Marital stability and the risk of domestic friction are heavily influenced by the spousal educational differential.
+1


Educational Parity vs. Disparity: Instead of raw education levels, the model needs categorical features indicating whether the husband is more educated, the wife is more educated, or the couple has educational parity.


Wife More Educated Flag: Women who achieve a higher educational milestone than their husbands deviate from traditional socialization and are significantly more likely to report marital instability and partner violence.
+1


Significant Educational Gap: It is also useful to mathematically flag if the husband has significantly more education than the wife (differing by at least two educational categories).

3. Religious & Cultural Heterogamy

Interfaith Marriage Flag: Marriages that cross socio-cultural lines face greater stress from internal value conflicts and external peer pressures. A binary feature indicating whether a woman married a man of a different religion serves as a strong predictor of instability.
+1

Inter-Caste Marriage: Extending the principle of endogamy/homogamy, couples crossing ethnic or caste lines generally face similar external societal pressures.

4. Early Marriage & Employment

Early Marriage Flag: Women who marry at an early age (under 18 years old) have a significantly higher chance of divorce compared to those who marry at 18 or older.


Employment Status: A woman's employment status directly impacts the duration and stability of the marriage, as financial independence alters the dynamics of marital dependency.

In [16]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

# -------------------------------------------------------------------
# 1. SOCIOLOGICAL FEATURE ENGINEERING FUNCTION
# Based on literature: Bennett et al., & Shita et al.
# -------------------------------------------------------------------
def engineer_demographic_features(df):
    """
    Transforms raw demographics into relational 'Friction Predictor' flags.
    Matches the exact columns from the provided sample data.
    """
    df_engineered = df.copy()
    
    # --- A. Age Heterogamy ---
    if 'Husband_age' in df_engineered.columns and "Wife's_current_age" in df_engineered.columns:
        # Raw difference
        df_engineered['Age_Gap'] = df_engineered['Husband_age'] - df_engineered["Wife's_current_age"]
        
        # High Age Gap (>= 7 years difference)
        df_engineered['High_Age_Gap_Flag'] = (df_engineered['Age_Gap'] >= 7).astype(int)
        
        # Wife Older (Reversal of traditional dynamic)
        df_engineered['Wife_Older_Flag'] = (df_engineered["Wife's_current_age"] > df_engineered['Husband_age']).astype(int)
        
        # Extreme Gap / Teen Marriage (Wife under 20, Husband >= 28)
        df_engineered['Extreme_Age_Gap_Flag'] = ((df_engineered["Wife's_current_age"] < 20) & (df_engineered['Husband_age'] >= 28)).astype(int)

    # --- B. Educational Heterogamy ---
    if "Husband's Education Level" in df_engineered.columns and "Wife's Education Level" in df_engineered.columns:
        # Educational Parity (Do they share the same education level?)
        df_engineered['Educational_Parity_Flag'] = (df_engineered["Husband's Education Level"] == df_engineered["Wife's Education Level"]).astype(int)

    # --- C. Socio-Cultural Heterogamy ---
    if 'Husband Religion' in df_engineered.columns and "Wife's Religion" in df_engineered.columns:
        # Interfaith Marriage
        df_engineered['Interfaith_Marriage_Flag'] = (df_engineered['Husband Religion'] != df_engineered["Wife's Religion"]).astype(int)
        
    if 'Husband Ethnicity' in df_engineered.columns and "Wife's Ethnicity" in df_engineered.columns:
        # Intercaste / Inter-Ethnicity Marriage
        df_engineered['Intercaste_Marriage_Flag'] = (df_engineered['Husband Ethnicity'] != df_engineered["Wife's Ethnicity"]).astype(int)

    # --- D. Economic / Agency Flags ---
    if "Wife's_occupation_(grouped)" in df_engineered.columns:
        # Working Wife Flag (Financial Independence alters dynamic)
        df_engineered['Wife_Working_Flag'] = (df_engineered["Wife's_occupation_(grouped)"] != 'Not working').astype(int)
        
    return df_engineered

# -------------------------------------------------------------------
# 2. DATA LOADING & TARGET CLUSTERING
# -------------------------------------------------------------------
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()

# Apply the Sociological Feature Engineering BEFORE splitting targets
df_features = engineer_demographic_features(df_features)

# Cluster the 9 survey targets into a single Marital Stability target
y_raw = df_features[target_cols].copy()
for col in target_cols:
    le = LabelEncoder()
    y_raw[col] = le.fit_transform(y_raw[col].astype(str))

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_features['Marital_Stability_Cluster'] = kmeans.fit_predict(y_raw)
single_target = 'Marital_Stability_Cluster'

value_counts = df_features[single_target].value_counts()
minority_class = value_counts.idxmin()
majority_class = value_counts.idxmax()

# -------------------------------------------------------------------
# 3. PREPARE MODEL INPUTS (Features & Missing Values)
# -------------------------------------------------------------------
feature_cols = [c for c in df_features.columns if c not in target_cols and c != single_target]
X = df_features[feature_cols].copy()
y = df_features[single_target].copy()

# Robust Missing Value Handling
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

X_encoded = pd.get_dummies(X, drop_first=True)

# -------------------------------------------------------------------
# 4. MODEL TRAINING WITH SCALE_POS_WEIGHT
# -------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Calculate weight ratio for the minority class to handle imbalance
ratio = float(np.sum(y_train == majority_class)) / np.sum(y_train == minority_class)
print(f"Calculated scale_pos_weight: {ratio:.2f}")

xgb = XGBClassifier(
    learning_rate=0.05,
    n_estimators=200,
    max_depth=6,
    scale_pos_weight=ratio,  # Forces model to respect the minority cluster
    eval_metric='logloss', 
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)

# -------------------------------------------------------------------
# 5. POST-TRAINING THRESHOLD OPTIMIZATION
# -------------------------------------------------------------------
# Extract probabilities for the high-risk minority class
y_probs = xgb.predict_proba(X_test)[:, minority_class] 

best_threshold = 0.50
best_f1 = -1
best_y_pred = []

# Sweep thresholds to find the perfect F1-Score (balance between Precision & Recall)
for thresh in np.arange(0.10, 0.90, 0.02):
    y_pred_temp = np.where(y_probs >= thresh, minority_class, majority_class)
    current_f1 = f1_score(y_test, y_pred_temp, pos_label=minority_class)
    
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_threshold = thresh
        best_y_pred = y_pred_temp

print(f"\n==================================================")
print(f"=== SOCIOLOGICAL ENGINEERED CLUSTER PERFORMANCE ===")
print(f"Minority Class (High Risk): {minority_class}")
print(f"Optimal Decision Threshold: {best_threshold*100:.0f}%")
print(f"Overall Accuracy: {accuracy_score(y_test, best_y_pred) * 100:.2f}%")
print(f"--------------------------------------------------")
print(classification_report(y_test, best_y_pred))

Calculated scale_pos_weight: 3.35

=== SOCIOLOGICAL ENGINEERED CLUSTER PERFORMANCE ===
Minority Class (High Risk): 1
Optimal Decision Threshold: 46%
Overall Accuracy: 48.72%
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.83      0.42      0.56      7206
           1       0.26      0.71      0.38      2092

    accuracy                           0.49      9298
   macro avg       0.55      0.57      0.47      9298
weighted avg       0.71      0.49      0.52      9298



We can extract the Feature Importances directly from the model. Furthermore, as the Predicting Marital Stability paper suggested, using simpler models like a Decision Tree or Naive Bayes often works better for sparse sociological data than complex algorithms like XGBoost.

In [17]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

# -------------------------------------------------------------------
# 1. ENGINEER FEATURES (From Bennett et al. & Shita et al.)
# -------------------------------------------------------------------
def engineer_demographic_features(df):
    df_engineered = df.copy()
    if 'Husband_age' in df_engineered.columns and "Wife's_current_age" in df_engineered.columns:
        df_engineered['Age_Gap'] = df_engineered['Husband_age'] - df_engineered["Wife's_current_age"]
        df_engineered['High_Age_Gap_Flag'] = (df_engineered['Age_Gap'] >= 7).astype(int) # Shita et al.
        df_engineered['Wife_Older_Flag'] = (df_engineered["Wife's_current_age"] > df_engineered['Husband_age']).astype(int) # Bennett et al.
        df_engineered['Extreme_Age_Gap_Flag'] = ((df_engineered["Wife's_current_age"] < 20) & (df_engineered['Husband_age'] >= 28)).astype(int)

    if "Husband's Education Level" in df_engineered.columns and "Wife's Education Level" in df_engineered.columns:
        df_engineered['Educational_Parity_Flag'] = (df_engineered["Husband's Education Level"] == df_engineered["Wife's Education Level"]).astype(int)

    if 'Husband Religion' in df_engineered.columns and "Wife's Religion" in df_engineered.columns:
        df_engineered['Interfaith_Marriage_Flag'] = (df_engineered['Husband Religion'] != df_engineered["Wife's Religion"]).astype(int)
        
    if 'Husband Ethnicity' in df_engineered.columns and "Wife's Ethnicity" in df_engineered.columns:
        df_engineered['Intercaste_Marriage_Flag'] = (df_engineered['Husband Ethnicity'] != df_engineered["Wife's Ethnicity"]).astype(int)

    if "Wife's_occupation_(grouped)" in df_engineered.columns:
        df_engineered['Wife_Working_Flag'] = (df_engineered["Wife's_occupation_(grouped)"] != 'Not working').astype(int)
        
    return df_engineered

# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men', 'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner', 'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner', 'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care", 'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()
df_features = engineer_demographic_features(df_features)

y_raw = df_features[target_cols].copy()
for col in target_cols:
    le = LabelEncoder()
    y_raw[col] = le.fit_transform(y_raw[col].astype(str))

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_features['Marital_Stability_Cluster'] = kmeans.fit_predict(y_raw)
single_target = 'Marital_Stability_Cluster'

feature_cols = [c for c in df_features.columns if c not in target_cols and c != single_target]
X = df_features[feature_cols].copy()
y = df_features[single_target].copy()

# Imputation & Encoding
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

if len(numeric_cols) > 0: X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])
if len(categorical_cols) > 0: X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

X_encoded = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# --- 2. TRAIN XGBOOST AND EXTRACT IMPORTANCES ---
value_counts = df_features[single_target].value_counts()
minority_class = value_counts.idxmin()
majority_class = value_counts.idxmax()
ratio = float(np.sum(y_train == majority_class)) / np.sum(y_train == minority_class)

xgb = XGBClassifier(learning_rate=0.05, n_estimators=200, max_depth=6, scale_pos_weight=ratio, random_state=42, n_jobs=-1)
xgb.fit(X_train, y_train)

print("==================================================")
print("TOP 10 MOST IMPORTANT SOCIOLOGICAL PREDICTORS")
print("==================================================")
importances = xgb.feature_importances_
feature_names = X_encoded.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)
for index, row in feature_importance_df.iterrows():
    print(f"{row['Feature']}: {row['Importance']:.4f}")

# --- 3. TRYING LITERATURE-RECOMMENDED SIMPLER MODELS ---
print("\n==================================================")
print("DECISION TREE CLASSIFIER (Highly Interpretable)")
print("==================================================")
# Decision Trees handle categorical flags very well and resist the "over-guessing" of XGBoost
dt = DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_dt) * 100:.2f}%")
print(classification_report(y_test, y_pred_dt))

print("\n==================================================")
print("NAIVE BAYES CLASSIFIER (Bayesian Probability)")
print("==================================================")
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_nb) * 100:.2f}%")
print(classification_report(y_test, y_pred_nb))

TOP 10 MOST IMPORTANT SOCIOLOGICAL PREDICTORS
State_Telangana: 0.0776
State_Nagaland: 0.0483
Wife's Religion_Christian: 0.0385
State_Karnataka: 0.0350
State_Jammu & Kashmir: 0.0346
State_Andhra Pradesh: 0.0249
State_Maharashtra: 0.0241
Husband Religion_Christian: 0.0224
Wife's Education Level_Secondary or below: 0.0205
State_Chhattisgarh: 0.0200

DECISION TREE CLASSIFIER (Highly Interpretable)
Overall Accuracy: 71.64%
              precision    recall  f1-score   support

           0       0.79      0.87      0.83      7206
           1       0.30      0.20      0.24      2092

    accuracy                           0.72      9298
   macro avg       0.55      0.53      0.53      9298
weighted avg       0.68      0.72      0.69      9298


NAIVE BAYES CLASSIFIER (Bayesian Probability)
Overall Accuracy: 41.48%
              precision    recall  f1-score   support

           0       0.83      0.31      0.45      7206
           1       0.25      0.79      0.38      2092

    accuracy   

The Strategy: SMOTETomek
SMOTETomek is a two-step mathematical process:

Oversampling (SMOTE): First, it generates synthetic data for your minority class to balance the scales.

Undersampling Noise (Tomek Links): Then, it scans the dataset for "Tomek Links"—pairs of data points from opposite classes that are extremely close to each other. It deletes these overlapping points.

In [19]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.combine import SMOTETomek # <-- THE NEW TECHNIQUE

warnings.filterwarnings('ignore')

# -------------------------------------------------------------------
# 1. ENGINEER FEATURES & PREPARE DATA
# -------------------------------------------------------------------
def engineer_demographic_features(df):
    df_engineered = df.copy()
    if 'Husband_age' in df_engineered.columns and "Wife's_current_age" in df_engineered.columns:
        df_engineered['Age_Gap'] = df_engineered['Husband_age'] - df_engineered["Wife's_current_age"]
        df_engineered['High_Age_Gap_Flag'] = (df_engineered['Age_Gap'] >= 7).astype(int)
        df_engineered['Wife_Older_Flag'] = (df_engineered["Wife's_current_age"] > df_engineered['Husband_age']).astype(int)
        df_engineered['Extreme_Age_Gap_Flag'] = ((df_engineered["Wife's_current_age"] < 20) & (df_engineered['Husband_age'] >= 28)).astype(int)

    if "Husband's Education Level" in df_engineered.columns and "Wife's Education Level" in df_engineered.columns:
        df_engineered['Educational_Parity_Flag'] = (df_engineered["Husband's Education Level"] == df_engineered["Wife's Education Level"]).astype(int)

    if 'Husband Religion' in df_engineered.columns and "Wife's Religion" in df_engineered.columns:
        df_engineered['Interfaith_Marriage_Flag'] = (df_engineered['Husband Religion'] != df_engineered["Wife's Religion"]).astype(int)
        
    if 'Husband Ethnicity' in df_engineered.columns and "Wife's Ethnicity" in df_engineered.columns:
        df_engineered['Intercaste_Marriage_Flag'] = (df_engineered['Husband Ethnicity'] != df_engineered["Wife's Ethnicity"]).astype(int)

    if "Wife's_occupation_(grouped)" in df_engineered.columns:
        df_engineered['Wife_Working_Flag'] = (df_engineered["Wife's_occupation_(grouped)"] != 'Not working').astype(int)
        
    return df_engineered

# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men', 'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner', 'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner', 'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care", 'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()
df_features = engineer_demographic_features(df_features)

y_raw = df_features[target_cols].copy()
for col in target_cols:
    le = LabelEncoder()
    y_raw[col] = le.fit_transform(y_raw[col].astype(str))

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_features['Marital_Stability_Cluster'] = kmeans.fit_predict(y_raw)
single_target = 'Marital_Stability_Cluster'

feature_cols = [c for c in df_features.columns if c not in target_cols and c != single_target]
X = df_features[feature_cols].copy()
y = df_features[single_target].copy()

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

if len(numeric_cols) > 0: X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])
if len(categorical_cols) > 0: X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

X_encoded = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# -------------------------------------------------------------------
# 2. APPLY SMOTE-TOMEK (The Magic Step)
# -------------------------------------------------------------------
print("Cleaning and balancing the data boundary with SMOTETomek...")
smt = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smt.fit_resample(X_train, y_train)

# -------------------------------------------------------------------
# 3. TRAIN MODELS ON CLEANED DATA
# -------------------------------------------------------------------
print("\n==================================================")
print("DECISION TREE (Trained on SMOTETomek Data)")
print("==================================================")
# We don't need class_weight='balanced' anymore because SMOTETomek balanced it perfectly
dt = DecisionTreeClassifier(max_depth=6, random_state=42)
dt.fit(X_train_resampled, y_train_resampled)
y_pred_dt = dt.predict(X_test)

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_dt) * 100:.2f}%")
print(classification_report(y_test, y_pred_dt))

print("\n==================================================")
print("XGBOOST (Trained on SMOTETomek Data)")
print("==================================================")
# Removed scale_pos_weight since data is balanced
xgb = XGBClassifier(learning_rate=0.05, n_estimators=150, max_depth=5, eval_metric='logloss', random_state=42, n_jobs=-1)
xgb.fit(X_train_resampled, y_train_resampled)
y_pred_xgb = xgb.predict(X_test)

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_xgb) * 100:.2f}%")
print(classification_report(y_test, y_pred_xgb))

Cleaning and balancing the data boundary with SMOTETomek...

DECISION TREE (Trained on SMOTETomek Data)
Overall Accuracy: 73.20%
              precision    recall  f1-score   support

           0       0.79      0.90      0.84      7206
           1       0.31      0.16      0.21      2092

    accuracy                           0.73      9298
   macro avg       0.55      0.53      0.52      9298
weighted avg       0.68      0.73      0.70      9298


XGBOOST (Trained on SMOTETomek Data)
Overall Accuracy: 73.34%
              precision    recall  f1-score   support

           0       0.79      0.89      0.84      7206
           1       0.33      0.19      0.24      2092

    accuracy                           0.73      9298
   macro avg       0.56      0.54      0.54      9298
weighted avg       0.69      0.73      0.70      9298



In [21]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import category_encoders as ce
from imblearn.combine import SMOTETomek

warnings.filterwarnings('ignore')

# 1. Load Data
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men', 'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner', 'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner', 'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care", 'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()

# --- CLUSTER THE TARGETS ---
y_raw = df_features[target_cols].copy()
for col in target_cols:
    le = LabelEncoder()
    y_raw[col] = le.fit_transform(y_raw[col].astype(str))

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_features['Marital_Stability_Cluster'] = kmeans.fit_predict(y_raw)
single_target = 'Marital_Stability_Cluster'

feature_cols = [c for c in df_features.columns if c not in target_cols and c != single_target]
X = df_features[feature_cols].copy()
y = df_features[single_target].copy()

# Split BEFORE Target Encoding to prevent severe data leakage!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- AUTOMATED FEATURE ENGINEERING ---
print("Initiating Automated Feature Engineering...")

numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object', 'category', 'str']).columns.tolist()

# 1. Impute Missing Values
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

X_train_num = pd.DataFrame(num_imputer.fit_transform(X_train[numeric_cols]), columns=numeric_cols, index=X_train.index)
X_test_num = pd.DataFrame(num_imputer.transform(X_test[numeric_cols]), columns=numeric_cols, index=X_test.index)

X_train_cat = pd.DataFrame(cat_imputer.fit_transform(X_train[categorical_cols]), columns=categorical_cols, index=X_train.index)
X_test_cat = pd.DataFrame(cat_imputer.transform(X_test[categorical_cols]), columns=categorical_cols, index=X_test.index)

# 2. TARGET ENCODING (For Categoricals)
# This replaces weak text labels with the actual mathematical risk probability
target_encoder = ce.TargetEncoder(cols=categorical_cols, smoothing=10) # Smoothing prevents overfitting
X_train_cat_encoded = target_encoder.fit_transform(X_train_cat, y_train)
X_test_cat_encoded = target_encoder.transform(X_test_cat)

# Combine for next steps
X_train_combined = pd.concat([X_train_num, X_train_cat_encoded], axis=1)
X_test_combined = pd.concat([X_test_num, X_test_cat_encoded], axis=1)

# 3. POLYNOMIAL INTERACTIONS (For Numericals)
# Automatically creates features like (Age^2) or (Age * Encoded_State)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train_combined)
X_test_poly = poly.transform(X_test_combined)

poly_feature_names = poly.get_feature_names_out(X_train_combined.columns)
X_train_auto = pd.DataFrame(X_train_poly, columns=poly_feature_names, index=X_train.index)
X_test_auto = pd.DataFrame(X_test_poly, columns=poly_feature_names, index=X_test.index)

# 4. LATENT DIMENSIONALITY (PCA)
# Compresses the massive interaction dataset into 10 "Hidden Super Features"
pca = PCA(n_components=10, random_state=42)
pca_train = pd.DataFrame(pca.fit_transform(X_train_auto), columns=[f'Latent_Feature_{i}' for i in range(1, 11)], index=X_train.index)
pca_test = pd.DataFrame(pca.transform(X_test_auto), columns=[f'Latent_Feature_{i}' for i in range(1, 11)], index=X_test.index)

# Final Feature Sets (Combining interactions and latent features)
X_train_final = pd.concat([X_train_auto, pca_train], axis=1)
X_test_final = pd.concat([X_test_auto, pca_test], axis=1)

print(f"Original Feature Count: {len(X_train.columns)}")
print(f"Auto-Engineered Feature Count: {len(X_train_final.columns)}")

# --- RESAMPLING & TRAINING ---
print("\nBalancing classes with SMOTETomek...")
smt = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smt.fit_resample(X_train_final, y_train)

print("Training XGBoost on Auto-Engineered Features...")
xgb = XGBClassifier(learning_rate=0.05, n_estimators=150, max_depth=5, eval_metric='logloss', random_state=42, n_jobs=-1)
xgb.fit(X_train_resampled, y_train_resampled)
y_pred_xgb = xgb.predict(X_test_final)

print("\n==================================================")
print("=== AUTO-ENGINEERED XGBOOST PERFORMANCE ===")
print("==================================================")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_xgb) * 100:.2f}%")
print(classification_report(y_test, y_pred_xgb))

Initiating Automated Feature Engineering...
Original Feature Count: 21
Auto-Engineered Feature Count: 241

Balancing classes with SMOTETomek...
Training XGBoost on Auto-Engineered Features...

=== AUTO-ENGINEERED XGBOOST PERFORMANCE ===
Overall Accuracy: 77.12%
              precision    recall  f1-score   support

           0       0.78      0.99      0.87      7206
           1       0.36      0.02      0.04      2092

    accuracy                           0.77      9298
   macro avg       0.57      0.50      0.45      9298
weighted avg       0.68      0.77      0.68      9298



In [15]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

warnings.filterwarnings('ignore')

# 1. Load Data (assuming df_features is ready)
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

df_features = df_features.dropna(subset=target_cols).copy()

# --- ENCODE THE 9 QUESTIONS ---
# These 9 questions will now become our predictive FEATURES (X)
X_survey = df_features[target_cols].copy()
label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    X_survey[col] = le.fit_transform(X_survey[col].astype(str))
    label_encoders[col] = le

# --- CLUSTER TO CREATE THE TARGET (y) ---
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
y_cluster = kmeans.fit_predict(X_survey)

# --- TRAIN-TEST SPLIT ---
X_train, X_test, y_train, y_test = train_test_split(X_survey, y_cluster, test_size=0.2, random_state=42)

print("==================================================")
print("1. BAYESIAN PREDICTION (NAÏVE BAYES)")
print("==================================================\n")

# Replicating the paper's Bayesian approach
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_nb) * 100:.2f}%\n")
print(classification_report(y_test, y_pred_nb))

print("==================================================")
print("2. DECISION TREE CLASSIFIER")
print("==================================================\n")

# Replicating the paper's Decision Tree approach
dt_model = DecisionTreeClassifier(max_depth=4, random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_dt) * 100:.2f}%\n")
print(classification_report(y_test, y_pred_dt))

1. BAYESIAN PREDICTION (NAÏVE BAYES)

Overall Accuracy: 93.42%

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      7206
           1       0.77      1.00      0.87      2092

    accuracy                           0.93      9298
   macro avg       0.89      0.96      0.91      9298
weighted avg       0.95      0.93      0.94      9298

2. DECISION TREE CLASSIFIER

Overall Accuracy: 99.81%

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7206
           1       1.00      0.99      1.00      2092

    accuracy                           1.00      9298
   macro avg       1.00      1.00      1.00      9298
weighted avg       1.00      1.00      1.00      9298



Stragetic Chain Ordering

In [10]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import ClassifierChain
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)

# 1. Load Data (assuming df_features is already loaded in your environment)
# df_features = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',                # Index 0
    'Husband_partner_insists_on_knowing_where_Wife_is',                    # Index 1
    'Ever_been_humiliated_by_husband_partner',                             # Index 2
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',           # Index 3
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',  # Index 4
    'Ever_been_slapped_by_husband_partner',                                # Index 5
    "Person_who_usually_decides_on_Wife's_health_care",                    # Index 6
    'Person_who_usually_decides_on_large_household_purchases',             # Index 7
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'  # Index 8
]

# Drop rows missing targets
df_features = df_features.dropna(subset=target_cols).copy()

# Feature engineering: Age Difference
if 'Husband_age' in df_features.columns and "Wife's_current_age" in df_features.columns:
    df_features['Age_Difference'] = df_features['Husband_age'] - df_features["Wife's_current_age"]

feature_cols = [c for c in df_features.columns if c not in target_cols]
X = df_features[feature_cols].copy()
y = df_features[target_cols].copy()

# --- Robust Missing Value Handling ---
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
if len(numeric_cols) > 0:
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

# Encode Categorical Features
X_encoded = pd.get_dummies(X, drop_first=True)

# Encode Targets
label_encoders = {}
for col in target_cols:
    le = LabelEncoder()
    y[col] = le.fit_transform(y[col].astype(str))
    label_encoders[col] = le

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# --- The Strategic XGBoost Setup ---
xgb = XGBClassifier(
    learning_rate=0.05,
    n_estimators=150,
    max_depth=5,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss', 
    random_state=42,
    n_jobs=-1
)

# Strategic Chain Order based on your previous accuracy results:
# Easiest -> Hardest
# Index 3: Insulted (92.8%)
# Index 2: Humiliated (91.8%)
# Index 4: Pushed/Shook (89.2%)
# Index 1: Insists knowing where wife is (80.6%)
# Index 5: Slapped (76.5%)
# Index 0: Jealous (74.9%)
# Index 6: Health care decider (74.1%)
# Index 7: Household purchases decider (74.1%)
# Index 8: Wife afraid (66.2%)
optimal_order = [3, 2, 4, 1, 5, 0, 6, 7, 8]

# Apply the custom order to the ClassifierChain
chain_xgb = ClassifierChain(xgb, order=optimal_order, random_state=42)

print("Training Strategically Ordered ClassifierChain XGBoost model...")
chain_xgb.fit(X_train, y_train)

# Predict and Evaluate
y_pred_test = chain_xgb.predict(X_test)

print("\n--- Test Accuracy per Survey Question ---")
test_accuracies = []
for i, col in enumerate(target_cols):
    acc = accuracy_score(y_test.iloc[:, i], y_pred_test[:, i])
    test_accuracies.append(acc)
    print(f"{col}: {acc * 100:.2f}%")

print(f"\n======================================")
print(f"Average Overall Test Accuracy: {np.mean(test_accuracies) * 100:.2f}%")
print(f"======================================")

Training Strategically Ordered ClassifierChain XGBoost model...

--- Test Accuracy per Survey Question ---
Husband_partner_jealous_if_Wife_talks_with_other_men: 74.94%
Husband_partner_insists_on_knowing_where_Wife_is: 80.67%
Ever_been_humiliated_by_husband_partner: 91.85%
Ever_been_insulted_or_made_to_feel_bad_by_husband_partner: 92.80%
Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner: 89.22%
Ever_been_slapped_by_husband_partner: 76.59%
Person_who_usually_decides_on_Wife's_health_care: 74.14%
Person_who_usually_decides_on_large_household_purchases: 74.12%
Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never: 66.10%

Average Overall Test Accuracy: 80.05%


USING THE DATASET BALANCING TECHNIQUES TO IMPROVE MODEL LEARNING:


In [1]:
import pandas as pd

import sys
print(sys.executable)

/home/codespace/.python/current/bin/python


In [4]:
df = pd.read_csv("completeresponse_dhs.csv")

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

print(len(df))

# --- PRINT FREQUENCIES FOR THE REVIEW BOARD ---
print("==================================================")
print("DATASET IMBALANCE (FREQUENCY PER QUESTION)")
print("==================================================\n")
for col in target_cols:
    print(df[col].value_counts())
    print("\n")

46488
DATASET IMBALANCE (FREQUENCY PER QUESTION)

Husband_partner_jealous_if_Wife_talks_with_other_men
No            34834
Yes           11498
Don't know      156
Name: count, dtype: int64


Husband_partner_insists_on_knowing_where_Wife_is
No            37603
Yes            8821
Don't know       64
Name: count, dtype: int64


Ever_been_humiliated_by_husband_partner
Never                                 42919
Sometimes                              2540
Often                                   592
Yes, but not in the last 12 months      437
Name: count, dtype: int64


Ever_been_insulted_or_made_to_feel_bad_by_husband_partner
Never                                 43320
Sometimes                              2258
Often                                   523
Yes, but not in the last 12 months      387
Name: count, dtype: int64


Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner
Never                                 41446
Sometimes                              3576
Yes, but no

In [5]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import resample
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

# 1. Load Data (assuming df is your raw dataset)
# df = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

# --- TARGET GROUPING / BINARIZATION ---
# We map the fragmented strings into a clean 0 (Stable) or 1 (High-Risk)
grouping_rules = {
    'Husband_partner_jealous_if_Wife_talks_with_other_men': {'No': 0, 'Yes': 1},
    'Husband_partner_insists_on_knowing_where_Wife_is': {'No': 0, 'Yes': 1},
    'Ever_been_humiliated_by_husband_partner': {'Never': 0, 'Sometimes': 1, 'Often': 1, 'Yes, but not in the last 12 months': 1},
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner': {'Never': 0, 'Sometimes': 1, 'Often': 1, 'Yes, but not in the last 12 months': 1},
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner': {'Never': 0, 'Sometimes': 1, 'Often': 1, 'Yes, but not in the last 12 months': 1},
    'Ever_been_slapped_by_husband_partner': {'Never': 0, 'Sometimes': 1, 'Often': 1, 'Yes, but not in the last 12 months': 1},
    "Person_who_usually_decides_on_Wife's_health_care": {'Respondent and husband/partner': 0, 'Respondent alone': 0, 'Husband/partner alone': 1},
    'Person_who_usually_decides_on_large_household_purchases': {'Respondent and husband/partner': 0, 'Respondent alone': 0, 'Husband/partner alone': 1},
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never': {'Never afraid': 0, 'Sometimes afraid': 1, 'Most of the time afraid': 1}
}

print("==================================================")
print("TRAINING INDEPENDENT BINARY MODELS WITH 1:1 UNDERSAMPLING")
print("==================================================\n")

# Loop through each question independently, as the majority/minority indices shift per question
for col in target_cols:
    print(f"\n--- Analyzing: {col} ---")
    
    # Extract current target and features
    df_current = df.copy()
    
    # Apply the grouping rules. Anything not in the dictionary (like "Don't know" or "Someone else") becomes NaN
    df_current[col] = df_current[col].map(grouping_rules[col])
    
    # Drop rows where the target is NaN (cleaning the "Don't knows")
    df_current = df_current.dropna(subset=[col])
    
    X = df_current.drop(columns=target_cols)
    y = df_current[col].astype(int)
    
    # Handle Features (Missing Values & Encoding)
    numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns

    num_imputer = SimpleImputer(strategy='median')
    cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

    if len(numeric_cols) > 0: X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])
    if len(categorical_cols) > 0: X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])
    
    X_encoded = pd.get_dummies(X, drop_first=True)
    
    # 1. Train-Test Split (80/20) BEFORE balancing
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)
    
    # 2. Extract Minority and Majority from the TRAINING set dynamically
    train_data = pd.concat([X_train, y_train], axis=1)
    
    # Let pandas figure out which is the bigger class
    class_counts = train_data[col].value_counts()
    majority_label = class_counts.idxmax()
    minority_label = class_counts.idxmin()
    
    majority_class = train_data[train_data[col] == majority_label]
    minority_class = train_data[train_data[col] == minority_label]
    
    # 3. Downsample Majority Class to match Minority length
    majority_downsampled = resample(majority_class, 
                                    replace=False,    # sample without replacement
                                    n_samples=len(minority_class), # exactly match minority length
                                    random_state=42)
    
    # 4. Combine into a perfectly balanced 1:1 training set
    train_balanced = pd.concat([majority_downsampled, minority_class])
    
    X_train_bal = train_balanced.drop(columns=[col])
    y_train_bal = train_balanced[col]
    
    print(f"Training Set Distribution: Class {majority_label}: {len(majority_downsampled)} | Class {minority_label}: {len(minority_class)}")
    
    # 5. Train XGBoost on the Balanced Data
    xgb = XGBClassifier(learning_rate=0.05, n_estimators=150, max_depth=5, eval_metric='logloss', random_state=42, n_jobs=-1)
    xgb.fit(X_train_bal, y_train_bal)
    
    # 6. Evaluate on the untouched, imbalanced Test Set
    y_pred = xgb.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {acc * 100:.2f}%")
    print(classification_report(y_test, y_pred, target_names=['Safe/Stable (0)', 'High Risk (1)'], zero_division=0))

TRAINING INDEPENDENT BINARY MODELS WITH 1:1 UNDERSAMPLING


--- Analyzing: Husband_partner_jealous_if_Wife_talks_with_other_men ---
Training Set Distribution: Class 0: 9198 | Class 1: 9198
Overall Accuracy: 61.98%
                 precision    recall  f1-score   support

Safe/Stable (0)       0.82      0.63      0.71      6967
  High Risk (1)       0.34      0.58      0.43      2300

       accuracy                           0.62      9267
      macro avg       0.58      0.61      0.57      9267
   weighted avg       0.70      0.62      0.64      9267


--- Analyzing: Husband_partner_insists_on_knowing_where_Wife_is ---
Training Set Distribution: Class 0: 7057 | Class 1: 7057
Overall Accuracy: 57.81%
                 precision    recall  f1-score   support

Safe/Stable (0)       0.86      0.57      0.69      7521
  High Risk (1)       0.25      0.60      0.35      1764

       accuracy                           0.58      9285
      macro avg       0.55      0.59      0.52      9285
   

In [54]:
df = pd.read_csv("completeresponse_dhs.csv")

In [55]:
import pandas as pd
import numpy as np
import warnings
import joblib # Library for saving models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler # <-- CHANGED TO RANDOM OVERSAMPLING

warnings.filterwarnings('ignore')

# 1. Load Data (assuming df is your raw dataset)
# df = pd.read_csv('your_actual_file.csv')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

# Drop rows missing targets
df_clean = df.dropna(subset=target_cols).copy()

# ==========================================
# PHASE 1: TARGET ENCODING & CLUSTERING
# ==========================================
print("Encoding targets and running K-Means Clustering...")
y_raw = df_clean[target_cols].copy()
label_encoders = {}

# Encode the 9 targets and save the encoders
for col in target_cols:
    le = LabelEncoder()
    y_raw[col] = le.fit_transform(y_raw[col].astype(str))
    label_encoders[col] = le

# Cluster into 2 groups (Stable vs Unstable)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
df_clean['Marital_Stability_Cluster'] = kmeans.fit_predict(y_raw)
single_target = 'Marital_Stability_Cluster'

# Dynamically map the minority cluster to '1' (High Risk) and majority to '0' (Stable)
class_counts = df_clean[single_target].value_counts()
majority_label = class_counts.idxmax()
minority_label = class_counts.idxmin()

# Remap to ensure XGBoost gets strict 0 and 1
df_clean[single_target] = df_clean[single_target].map({majority_label: 0, minority_label: 1})

print(f"Dataset Binarized: 0 (Stable) = {class_counts.max()} cases | 1 (Unstable/Risk) = {class_counts.min()} cases")

# ==========================================
# PHASE 2: FEATURE PREPARATION & SPLITTING
# ==========================================
feature_cols = [c for c in df_clean.columns if c not in target_cols and c != single_target]
X = df_clean[feature_cols].copy()
y = df_clean[single_target].copy()

# Handle Missing Values
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

if len(numeric_cols) > 0: X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])
if len(categorical_cols) > 0: X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

# Dummy Encode Categorical Features
X_encoded = pd.get_dummies(X, drop_first=True)

# SPLIT FIRST to prevent Data Leakage!
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

# ==========================================
# PHASE 3: RANDOM OVERSAMPLING
# ==========================================
print("\nOversampling minority class in the TRAINING set using Random Oversampling...")
ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

print(f"Balanced Training Set Distribution: 0: {sum(y_train_bal==0)} | 1: {sum(y_train_bal==1)}")

# ==========================================
# PHASE 4: MODEL TRAINING & EVALUATION
# ==========================================
print("\nTraining XGBoost on Balanced Data...")
xgb = XGBClassifier(learning_rate=0.05, n_estimators=150, max_depth=5, eval_metric='logloss', random_state=42, n_jobs=-1)
xgb.fit(X_train_bal, y_train_bal)

y_pred = xgb.predict(X_test)

print("\n==================================================")
print("=== FINAL MODEL PERFORMANCE ON UNTOUCHED TEST SET ===")
print("==================================================")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred, target_names=['Safe/Stable (0)', 'High Risk (1)']))

# ==========================================
# PHASE 5: SAVING THE MODELS
# ==========================================
print("\nSaving Models to Disk...")

# Save K-Means Model
joblib.dump(kmeans, 'kmeans_cluster_model.pkl')
# Save Target Label Encoders
joblib.dump(label_encoders, 'target_label_encoders.pkl')
# Save the XGBoost Model
joblib.dump(xgb, 'xgboost_binary_model.pkl')

print("Success! Models saved as .pkl files.")

Encoding targets and running K-Means Clustering...
Dataset Binarized: 0 (Stable) = 35844 cases | 1 (Unstable/Risk) = 10644 cases

Oversampling minority class in the TRAINING set using Random Oversampling...
Balanced Training Set Distribution: 0: 28675 | 1: 28675

Training XGBoost on Balanced Data...

=== FINAL MODEL PERFORMANCE ON UNTOUCHED TEST SET ===
Overall Accuracy: 59.58%
                 precision    recall  f1-score   support

Safe/Stable (0)       0.82      0.61      0.70      7169
  High Risk (1)       0.30      0.56      0.39      2129

       accuracy                           0.60      9298
      macro avg       0.56      0.58      0.54      9298
   weighted avg       0.70      0.60      0.63      9298


Saving Models to Disk...
Success! Models saved as .pkl files.


Cluster profiling: Proof that K-Means cluster grouping the stable and unstable class based on 
positive and negative ansers in the 9 survey questions

Phase 1: Overcoming Algorithmic Bias in Target Creation
The Unsupervised Learning Trap (K-Means Clustering)
Initially, an unsupervised K-Means clustering algorithm was deployed to dynamically segment the dataset into two underlying classes based on the 9 marital survey questions. However, subsequent Cluster Profiling (Centroid Analysis) using cross-tabulation exposed a critical flaw: the algorithm fell into a "Variance Trap."

Because instances of extreme physical abuse are statistically rarer than instances of financial inequality, the K-Means algorithm optimized for the wrong dimension. It grouped women who had never been slapped together with women who had been slapped "Often," simply because they shared similar household purchasing dynamics. Unsupervised math failed to distinguish between lethal red flags and mundane relationship friction.

In [56]:
import pandas as pd

# Assuming df_clean already has the 'Marital_Stability_Cluster' column attached from your K-Means step
# single_target = 'Marital_Stability_Cluster'

print("==================================================")
print("CLUSTER PROFILING: MATHEMATICAL PROOF OF LABELS")
print("==================================================\n")

# Let's look at a few highly critical questions to prove the difference
critical_questions = [
    'Ever_been_slapped_by_husband_partner',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never',
    "Person_who_usually_decides_on_Wife's_health_care"
]

for col in critical_questions:
    print(f"--- Distribution for: {col} ---")
    
    # Create a cross-tabulation showing the percentage of each answer within the two clusters
    crosstab = pd.crosstab(
        index=df_clean['Marital_Stability_Cluster'], 
        columns=df_clean[col], 
        normalize='index' # Converts counts to percentages per cluster
    ) * 100
    
    # Format for clean printing
    print(crosstab.round(1).to_string())
    print("\n")

CLUSTER PROFILING: MATHEMATICAL PROOF OF LABELS

--- Distribution for: Ever_been_slapped_by_husband_partner ---
Ever_been_slapped_by_husband_partner  Never  Often  Sometimes  Yes, but not in the last 12 months
Marital_Stability_Cluster                                                                        
0                                      78.8    1.4       15.0                                 4.8
1                                      69.9    3.2       20.6                                 6.3


--- Distribution for: Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never ---
Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never  Most of the time afraid  Never afraid  Sometimes afraid
Marital_Stability_Cluster                                                                                                   
0                                                                                        8.6          25.3              66.1
1                        

The Shift to Domain-Driven Determinism
To mathematically guarantee that the target variable accurately reflected marital danger, the methodology was pivoted to a Deterministic Rule-Based Composite Target.

Drawing directly from the sociological severity of the survey, a strict operational definition was established:

Class 1 (High-Risk/Unstable): Any demographic profile where the respondent reported any instance of physical violence, extreme fear, or complete loss of healthcare/financial agency (e.g., answering "Often," "Sometimes," or "Yes").

Class 0 (Stable): A strict baseline requiring absolute zero red flags across all 9 survey dimensions.

By explicitly prioritizing the severe abuse indicators, the target variable was successfully tethered to the reality of the dataset rather than blind algorithmic variance.

In [57]:
import pandas as pd
import numpy as np

# 1. Define the Red Flag answers for ALL 9 questions
red_flag_answers = [
    'Yes', 
    'Often', 
    'Sometimes', 
    'Most of the time afraid',
    'Husband/partner alone',
]

# 2. The function that reads a woman's entire survey profile at once
def determine_risk(row):
    # We put ALL 9 target columns here
    all_9_targets = [
        'Husband_partner_jealous_if_Wife_talks_with_other_men',
        'Husband_partner_insists_on_knowing_where_Wife_is',
        'Ever_been_humiliated_by_husband_partner',
        'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
        'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
        'Ever_been_slapped_by_husband_partner',
        "Person_who_usually_decides_on_Wife's_health_care",
        'Person_who_usually_decides_on_large_household_purchases',
        'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
    ]
    
    # Check every single question for this specific woman
    for col in all_9_targets:
        if row[col] in red_flag_answers:
            return 1 # If she answered ANY red flag, mark her marriage as High Risk (1)
            
    return 0 # If she has ZERO red flags across all 9 questions, mark as Stable (0)

# 3. Apply it to the entire dataframe at once!
# axis=1 means "read across the row"
df_clean['Marital_Stability_Target'] = df_clean.apply(determine_risk, axis=1)

# Print the final, single binary distribution
print(df_clean['Marital_Stability_Target'].value_counts())

Marital_Stability_Target
1    25987
0    20501
Name: count, dtype: int64


In [58]:
# Assuming df_clean already has the newly created 'Marital_Stability_Target'
print("==================================================")
print("POST-HOC PROFILING: PROOF OF DETERMINISTIC TARGET")
print("==================================================\n")

critical_questions = [
    'Ever_been_slapped_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never',
    "Person_who_usually_decides_on_Wife's_health_care"
]

for col in critical_questions:
    print(f"--- Distribution for: {col} ---")
    
    # Create the cross-tabulation table
    crosstab = pd.crosstab(
        index=df_clean['Marital_Stability_Target'], 
        columns=df_clean[col], 
        normalize='index' # Converts to percentage per cluster
    ) * 100
    
    print(crosstab.round(1).to_string())
    print("\n")

POST-HOC PROFILING: PROOF OF DETERMINISTIC TARGET

--- Distribution for: Ever_been_slapped_by_husband_partner ---
Ever_been_slapped_by_husband_partner  Never  Often  Sometimes  Yes, but not in the last 12 months
Marital_Stability_Target                                                                         
0                                      95.9    0.0        0.0                                 4.1
1                                      61.6    3.3       29.1                                 6.0


--- Distribution for: Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner ---
Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner  Never  Often  Sometimes  Yes, but not in the last 12 months
Marital_Stability_Target                                                                                                       
0                                                                    98.9    0.0        0.0                                 1.1
1              

In [59]:
import pandas as pd
import numpy as np
import warnings
import joblib 
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler

warnings.filterwarnings('ignore')

# 1. Prepare Features (X) and Target (y)
# (Assuming df_clean already contains the 9 target columns and the new 'Marital_Stability_Target')

target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

# Drop the 9 original target questions, keeping only demographics + the new composite target
X = df_clean.drop(columns=target_cols + ['Marital_Stability_Target'])
y = df_clean['Marital_Stability_Target']

# 2. Handle Missing Values and Encode Categorical Demographics
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

if len(numeric_cols) > 0: 
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])
if len(categorical_cols) > 0: 
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

# Dummy Encode Categorical Features
X_encoded = pd.get_dummies(X, drop_first=True)

# 3. SPLIT FIRST to prevent Data Leakage
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

print("Original Training Set Distribution:")
print(y_train.value_counts())

# 4. RANDOM OVERSAMPLING (Only on Training Data)
print("\nBalancing the training set using Random Oversampling...")
ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

print("\nBalanced Training Set Distribution:")
print(y_train_bal.value_counts())

# 5. TRAIN XGBOOST
print("\nTraining XGBoost Classifier...")
xgb = XGBClassifier(
    learning_rate=0.03, 
    n_estimators=600, 
    max_depth=6, 
    eval_metric='logloss', 
    random_state=42, 
    n_jobs=-1
)
xgb.fit(X_train_bal, y_train_bal)

# 6. EVALUATE ON UNTOUCHED TEST SET
y_pred = xgb.predict(X_test)

print("\n==================================================")
print("=== FINAL DETERMINISTIC MODEL PERFORMANCE ===")
print("==================================================")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%\n")
print(classification_report(y_test, y_pred, target_names=['Safe/Stable (0)', 'High Risk (1)']))

# 7. SAVE THE DEPLOYMENT MODEL
joblib.dump(xgb, 'final_xgboost_deterministic_model.pkl')
# Save the exact feature columns so your deployment tool knows the required input structure
joblib.dump(X_encoded.columns.tolist(), 'model_feature_columns.pkl')

print("\nModel and feature columns successfully saved to disk as .pkl files.")

# ==================================================
# 8. GENERATE AND SAVE DECODED BALANCED DATASET
# ==================================================
print("\n==================================================")
print("=== GENERATING DECODED BALANCED DATASET ===")
print("==================================================")

# X is our imputed but UNENCODED feature set. 
# We apply the exact same split seed to get the exact same rows.
X_train_txt, X_test_txt, y_train_txt, y_test_txt = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Apply the exact same RandomOverSampler seed
ros_txt = RandomOverSampler(random_state=42)
X_train_bal_txt, y_train_bal_txt = ros_txt.fit_resample(X_train_txt, y_train_txt)

# Combine the unencoded features and the target into one clean DataFrame
decoded_export_df = X_train_bal_txt.copy()
decoded_export_df['Marital_Stability_Target'] = y_train_bal_txt

# Save to CSV
csv_filename = 'Decoded_Balanced_Training_Data.csv'
decoded_export_df.to_csv(csv_filename, index=False)
print(f"Success! Decoded training data saved as '{csv_filename}'.")
print("You can now download this file from your Codespace Explorer for the T-Test and Chi-Square audit.")

Original Training Set Distribution:
Marital_Stability_Target
1    20789
0    16401
Name: count, dtype: int64

Balancing the training set using Random Oversampling...

Balanced Training Set Distribution:
Marital_Stability_Target
1    20789
0    20789
Name: count, dtype: int64

Training XGBoost Classifier...

=== FINAL DETERMINISTIC MODEL PERFORMANCE ===
Overall Accuracy: 69.35%

                 precision    recall  f1-score   support

Safe/Stable (0)       0.61      0.85      0.71      4100
  High Risk (1)       0.83      0.57      0.68      5198

       accuracy                           0.69      9298
      macro avg       0.72      0.71      0.69      9298
   weighted avg       0.73      0.69      0.69      9298


Model and feature columns successfully saved to disk as .pkl files.

=== GENERATING DECODED BALANCED DATASET ===
Success! Decoded training data saved as 'Decoded_Balanced_Training_Data.csv'.
You can now download this file from your Codespace Explorer for the T-Test and Chi

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

print("==================================================")
print("GENERATING DECODED BALANCED DATASET")
print("==================================================\n")

# Assuming 'X' is your features dataframe right BEFORE you applied pd.get_dummies()
# and 'y' is your Marital_Stability_Target
X_unencoded = X.copy()

# 1. Split the unencoded text data using the exact same seed (42)
X_train_txt, X_test_txt, y_train_txt, y_test_txt = train_test_split(
    X_unencoded, y, test_size=0.2, random_state=42, stratify=y
)

# 2. Oversample the unencoded text data using the exact same seed (42)
ros = RandomOverSampler(random_state=42)
X_train_bal_txt, y_train_bal_txt = ros.fit_resample(X_train_txt, y_train_txt)

# 3. Combine and Save to CSV
decoded_export_df = X_train_bal_txt.copy()
decoded_export_df['Marital_Stability_Target'] = y_train_bal_txt

decoded_export_df.to_csv('Balanced_Training_Data.csv', index=False)
print("Success! 'Balanced_Training_Data.csv' saved to your folder.")

GENERATING DECODED BALANCED DATASET

Success! 'Balanced_Training_Data.csv' saved to your folder.


Only Rule based target derivation without clustering ---> 

In [25]:
import pandas as pd
import numpy as np
import warnings
import joblib 
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler

warnings.filterwarnings('ignore')

print("==================================================")
print("PHASE 1: DATA LOADING & TARGET DERIVATION")
print("==================================================\n")

# 1. Load the raw data
# Replace with your actual file path if different
df = pd.read_csv('completeresponse_dhs.csv')

# The 9 original survey questions
target_cols = [
    'Husband_partner_jealous_if_Wife_talks_with_other_men',
    'Husband_partner_insists_on_knowing_where_Wife_is',
    'Ever_been_humiliated_by_husband_partner',
    'Ever_been_insulted_or_made_to_feel_bad_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Ever_been_slapped_by_husband_partner',
    "Person_who_usually_decides_on_Wife's_health_care",
    'Person_who_usually_decides_on_large_household_purchases',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never'
]

# Drop rows where any of the critical survey answers are missing to ensure clean rule processing
df_clean = df.dropna(subset=target_cols).copy()

# 2. Define the exact red-flag strings (Updated to catch all variations)
red_flag_answers = [
    'Yes', 
    'Often', 
    'Sometimes', 
    'Yes, but not in the last 12 months', 
    'Most of the time afraid',
    'Sometimes afraid',                   
    'Husband/partner alone' 
]

# 3. Create the Deterministic Rule Function
def determine_risk(row):
    for col in target_cols:
        if row[col] in red_flag_answers:
            return 1 # High Risk / Unstable (At least one red flag)
    return 0 # Stable (Zero red flags)

# Apply the rule to create the single binary target
df_clean['Marital_Stability_Target'] = df_clean.apply(determine_risk, axis=1)

print("Target Derivation Complete. Class Distribution:")
print(df_clean['Marital_Stability_Target'].value_counts())

print("\n==================================================")
print("PHASE 2: FEATURE PREPARATION & SPLITTING")
print("==================================================\n")

# Drop the original 9 survey questions; keep demographics and the new target
X = df_clean.drop(columns=target_cols + ['Marital_Stability_Target'])
y = df_clean['Marital_Stability_Target']

# Identify column types
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category', 'str']).columns

# Impute missing values
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

if len(numeric_cols) > 0: 
    X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])
if len(categorical_cols) > 0: 
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

# Dummy encode categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# STRATIFIED SPLIT FIRST to prevent Data Leakage
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training Set Dimensions before balancing: {X_train.shape}")

print("\n==================================================")
print("PHASE 3: RANDOM OVERSAMPLING & MODEL TRAINING")
print("==================================================\n")

print("Balancing the minority class in the training set...")
ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

print("Balanced Training Class Distribution:")
print(y_train_bal.value_counts())

print("\nTraining XGBoost Classifier...")
xgb = XGBClassifier(
    learning_rate=0.03, 
    n_estimators=600, 
    max_depth=6, 
    eval_metric='logloss', 
    random_state=42, 
    n_jobs=-1
)
xgb.fit(X_train_bal, y_train_bal)

print("\n==================================================")
print("PHASE 4: FINAL EVALUATION ON UNTOUCHED TEST SET")
print("==================================================\n")

y_pred = xgb.predict(X_test)

print(f"Overall Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%\n")
print(classification_report(y_test, y_pred, target_names=['Stable / Safe (0)', 'High Risk / Unstable (1)']))

# Save the final artifacts for deployment
joblib.dump(xgb, 'final_xgboost_model.pkl')
joblib.dump(X_encoded.columns.tolist(), 'model_feature_columns.pkl')

print("Success! Final model and feature schema saved to disk.")

PHASE 1: DATA LOADING & TARGET DERIVATION

Target Derivation Complete. Class Distribution:
Marital_Stability_Target
1    39877
0     6611
Name: count, dtype: int64

PHASE 2: FEATURE PREPARATION & SPLITTING

Training Set Dimensions before balancing: (37190, 84)

PHASE 3: RANDOM OVERSAMPLING & MODEL TRAINING

Balancing the minority class in the training set...
Balanced Training Class Distribution:
Marital_Stability_Target
0    31901
1    31901
Name: count, dtype: int64

Training XGBoost Classifier...

PHASE 4: FINAL EVALUATION ON UNTOUCHED TEST SET

Overall Accuracy: 68.26%

                          precision    recall  f1-score   support

       Stable / Safe (0)       0.24      0.56      0.33      1322
High Risk / Unstable (1)       0.91      0.70      0.79      7976

                accuracy                           0.68      9298
               macro avg       0.57      0.63      0.56      9298
            weighted avg       0.81      0.68      0.73      9298

Success! Final model 

In [26]:
# Assuming df_clean already has the newly created 'Marital_Stability_Target'
print("==================================================")
print("POST-HOC PROFILING: PROOF OF DETERMINISTIC TARGET")
print("==================================================\n")

critical_questions = [
    'Ever_been_slapped_by_husband_partner',
    'Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner',
    'Wife_afraid_of_husband_partner_most_of_the_time,_sometimes_or_never',
    "Person_who_usually_decides_on_Wife's_health_care"
]

for col in critical_questions:
    print(f"--- Distribution for: {col} ---")
    
    # Create the cross-tabulation table
    crosstab = pd.crosstab(
        index=df_clean['Marital_Stability_Target'], 
        columns=df_clean[col], 
        normalize='index' # Converts to percentage per cluster
    ) * 100
    
    print(crosstab.round(1).to_string())
    print("\n")

POST-HOC PROFILING: PROOF OF DETERMINISTIC TARGET

--- Distribution for: Ever_been_slapped_by_husband_partner ---
Ever_been_slapped_by_husband_partner  Never  Often  Sometimes  Yes, but not in the last 12 months
Marital_Stability_Target                                                                         
0                                     100.0    0.0        0.0                                 0.0
1                                      72.9    2.1       19.0                                 6.0


--- Distribution for: Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner ---
Ever_been_pushed,_shook_or_had_something_thrown_by_husband_partner  Never  Often  Sometimes  Yes, but not in the last 12 months
Marital_Stability_Target                                                                                                       
0                                                                   100.0    0.0        0.0                                 0.0
1              

In [64]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency
import warnings

warnings.filterwarnings('ignore')

print("==================================================")
print("LOADING DATASETS FOR GLOBAL STATISTICAL AUDIT")
print("==================================================\n")

# 1. Load the original raw dataset
df_orig = pd.read_csv('completeresponse_dhs.csv')

# 2. Load the DECODED balanced training dataset
df_bal = pd.read_csv('Decoded_Balanced_Training_Data.csv')

print("==================================================")
print("GLOBAL STATISTICAL PROOF (T-TEST & CHI-SQUARE)")
print("==================================================\n")

results = []

# --- 1. INDEPENDENT T-TEST FOR CONTINUOUS VARIABLES ---
continuous_cols = ["Husband_age", "Wife's_current_age", "Wife's height(centimeters)"]

print("Running T-Tests on Continuous Demographics...")
for col in continuous_cols:
    # Drop NAs to avoid mathematical errors during mean calculations
    orig_data = df_orig[col].dropna()
    bal_data = df_bal[col].dropna()
    
    # T-Test checks if the overall mathematical mean shifted globally
    stat, p_value = ttest_ind(orig_data, bal_data, equal_var=True)
    results.append({'Feature': col, 'Test Type': 'T-Test', 'p-value': p_value})


# --- 2. CHI-SQUARE TEST FOR CATEGORICAL VARIABLES ---
categorical_cols = [
    "Husband's Education Level", "Husband Occupation_(grouped)", "Husband Ethnicity", 
    "Husband Religion", "Wife's Education Level", "Wife's_occupation_(grouped)", 
    "Wife's Ethnicity", "Wife's Religion", "residence", "State"
]

print("Running Chi-Square Tests on Categorical Demographics...")
for col in categorical_cols:
    # Directly count the global text categories in both entire datasets
    orig_counts = df_orig[col].value_counts()
    bal_counts = df_bal[col].value_counts()
    
    # Align the counts into a single dataframe (contingency table)
    contingency_df = pd.DataFrame({'Original': orig_counts, 'Balanced': bal_counts}).fillna(0)
    
    # Run the Chi-Square test on the global text distributions
    stat, p_value, dof, expected = chi2_contingency(contingency_df.T)
    results.append({'Feature': col, 'Test Type': 'Chi-Square', 'p-value': p_value})

# --- DISPLAY THE FINAL PROOF ---
proof_df = pd.DataFrame(results)
proof_df['p-value'] = proof_df['p-value'].round(4)

# p-value > 0.05 proves the global distributions are statistically identical
proof_df['Conclusion'] = np.where(proof_df['p-value'] > 0.05, 'Distributions Identical', 'Distributions Altered')

print("\n--- Global Statistical Hypothesis Test Results ---")
print(proof_df.to_string(index=False))

LOADING DATASETS FOR GLOBAL STATISTICAL AUDIT

GLOBAL STATISTICAL PROOF (T-TEST & CHI-SQUARE)

Running T-Tests on Continuous Demographics...
Running Chi-Square Tests on Categorical Demographics...

--- Global Statistical Hypothesis Test Results ---
                     Feature  Test Type  p-value              Conclusion
                 Husband_age     T-Test   0.5927 Distributions Identical
          Wife's_current_age     T-Test   0.4224 Distributions Identical
  Wife's height(centimeters)     T-Test   0.3073 Distributions Identical
   Husband's Education Level Chi-Square   0.5008 Distributions Identical
Husband Occupation_(grouped) Chi-Square   0.7545 Distributions Identical
           Husband Ethnicity Chi-Square   0.7599 Distributions Identical
            Husband Religion Chi-Square   0.9896 Distributions Identical
      Wife's Education Level Chi-Square   0.3875 Distributions Identical
 Wife's_occupation_(grouped) Chi-Square   0.9935 Distributions Identical
            Wife's Et